# FastAPI upload server (payload_video.ipynb)

Notebook ini menyediakan server FastAPI yang menerima upload video (multipart) di `/upload` dan menerima JSON payload di `/upload`.

Langkah eksekusi:
1. Jalankan cell instalasi dependensi
2. Jalankan cell setup direktori
3. Jalankan cell definisi server
4. Jalankan cell start server (ngrok akan dicoba jika tersedia)

Hasil: file yang diupload akan disimpan di folder `uploads/` dan payload JSON yang dikirim ke `/upload` akan disimpan di `received_payloads/`. Video akan diproses dengan Whisper untuk speech-to-text.

In [ ]:
#kalo pake colab jangan lupa install dulu di terminal
!pip install --quiet numpy==1.26.4
!pip install --quiet --upgrade torch torchaudio faster-whisper


#kalo lokal download ffmpeg nya
#https://github.com/GyanD/codexffmpeg/releases/download/2025-11-27-git-61b034a47c/ffmpeg-2025-11-27-git-61b034a47c-full_build.zip
#simpen di c

In [ ]:
# import os
# os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"

# ============================================================================
# 🔧 CELL 1: INSTALL SAFE DEPENDENCIES (FIXED - NO CONFLICTS!)
# ============================================================================

# ✅ TIER 0: JUPYTER WIDGETS (fixes tqdm warning)
!pip install --quiet ipywidgets jupyter
# ✅ TIER 1: AMAN (Tidak touch numpy)
!pip install --quiet fastapi uvicorn nest-asyncio pyngrok python-multipart
!pip install --quiet tqdm
!pip install --quiet imageio-ffmpeg
!pip install --quiet deepl

# ✅ TIER 2: AMAN (Pure torch-based, no numpy dependency)
#!pip install --quiet torch torchaudio
!pip install --quiet silero-vad

# ✅ TIER 3: AMAN (Minimal numpy, tidak upgrade)
!pip install --quiet pydub
!pip install --quiet soundfile
!pip install --quiet scipy
!pip install --quiet scikit-learn

# ✅ TIER 4: AMAN (Cloud-based, no local deps)
#!pip install --quiet faster-whisper
!pip install --quiet huggingface-hub

# ✅ TIER 5: MEDIAPIPE (sudah include opencv internally!)
!pip install --quiet mediapipe
# ✅ TIER 6: TORCHCODEC (video codec support)
!pip install --quiet torchcodec
!pip install --quiet librosa

!pip install --quiet gdown requests

print('\n✅ All safe packages installed')
print('   ✅ No numpy version conflicts')
print('   ✅ Jupyter widgets installed (fixes tqdm warning)')
print('   ✅ FFmpeg required for audio - verify with next cell')

In [ ]:
# !pip uninstall torch torchaudio -y
# !pip cache purge
# !pip install torch torchaudio --no-cache-dir

<b><h2> Import Library

In [ ]:
# ==========================
# Standard Library
# ==========================
import asyncio
import gc
import getpass
import hashlib
import json
import json as json_module
import os
import random
import re
import shutil
import subprocess
import sys
import tempfile
import threading
import threading as th
import time
import math
import traceback
import uuid
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timezone
from typing import List
from urllib.parse import urlparse
import urllib.request
import torch
import torchaudio
from silero_vad import load_silero_vad
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import gdown
import requests
from urllib.parse import urlparse, parse_qs
import librosa

# ==========================
# Third-Party Libraries
# ==========================
import deepl
import nest_asyncio
import torch
import uvicorn
from faster_whisper import WhisperModel
from huggingface_hub import InferenceClient
from pyngrok import conf, ngrok
from tqdm import tqdm
import cv2
import mediapipe as mp

# ==========================
# FastAPI & Middleware
# ==========================
from fastapi import (
    BackgroundTasks,
    FastAPI,
    File,
    Form,
    HTTPException,
    Request,
    UploadFile
)
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles

<b><h2> Siapkan direktori untuk upload dan transcription

In [ ]:
# Siapkan direktori untuk upload dan transcription
ROOT_DIR = os.getcwd()
UPLOAD_DIR = os.path.join(ROOT_DIR, 'uploads')
TRANSCRIPTION_DIR = os.path.join(ROOT_DIR, 'transcriptions')
AUDIO_DIR = os.path.join(ROOT_DIR, 'audio')
RESULTS_DIR = os.path.join(ROOT_DIR, 'results')  # NEW: hasil assessment
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print('📁 Directories:')
print(f'   Upload: {UPLOAD_DIR}')
print(f'   Transcription: {TRANSCRIPTION_DIR}')
print(f'   AUDIO: {AUDIO_DIR}')
print(f'   Results: {RESULTS_DIR}')

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'\n🎯 Device Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')
if device == "cuda":
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
else:
    print('   Note: Using CPU (GPU recommended for faster processing)')

# DeepL Configuration
DEEPL_API_KEY = "02a88edf-4fcb-4786-ba3d-a137fb143760:fx"

print('\n🌐 Translation Configuration:')
print(f'   DeepL API: {"Configured" if DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE" else "⚠️  NOT CONFIGURED - Set DEEPL_API_KEY"}')

In [ ]:
app = FastAPI(title='AI Interview Assessment System')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
    expose_headers=['*'],
    max_age=3600,
)

# Mount static folders
app.mount('/uploads', StaticFiles(directory=UPLOAD_DIR), name='uploads')
app.mount('/transcriptions', StaticFiles(directory=TRANSCRIPTION_DIR), name='transcriptions')
app.mount('/results', StaticFiles(directory=RESULTS_DIR), name='results')

In [ ]:
# Background processing
executor = ThreadPoolExecutor(max_workers=2)
processing_status = {}
processing_lock = th.Lock()

# HELPER FUNCTIONS - ONLY ONE INSTANCE EACH

def get_local_file_path(url):
    """Extract local file path from URL if it's a local upload"""
    try:
        parsed = urlparse(url)
        if '/uploads/' in parsed.path:
            filename = parsed.path.split('/uploads/')[-1]
            local_path = os.path.join(UPLOAD_DIR, filename)
            if os.path.exists(local_path):
                return local_path
    except Exception as e:
        print(f'Error parsing URL: {e}')
    return None

<b><h2> **Initialize** Whisper Model

In [ ]:
# Load faster-whisper model with BEST ACCURACY settings
print('\n📥 Loading Whisper model...')
print('ℹ️  Using faster-whisper "large-v3" model')
print('   This is the MOST ACCURATE model available')
print('   Speed: 4-5x faster than openai-whisper')
print('   Accuracy: ~98% for clear English speech')
print('   First run will download ~3GB model...\n')

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'🎯 Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')

# Load model with best accuracy settings
whisper_model = WhisperModel(
    "large-v3",
    device=device,
    compute_type=compute_type,
    cpu_threads=4,
    num_workers=1
)

print('✅ Whisper model loaded successfully\n')

<b><h2> Initialize DeepL translator

In [ ]:
# Initialize DeepL translator
translator = None
if DEEPL_API_KEY and DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE":
    try:
        translator = deepl.Translator(DEEPL_API_KEY)
        print('✅ DeepL translator initialized successfully\n')
    except Exception as e:
        print(f'⚠️  DeepL initialization failed: {e}')
        print('   Translation to Indonesian will be skipped\n')
else:
    print('⚠️  DeepL API key not configured')
    print('   Translation to Indonesian will be skipped\n')

<b><h2> Fungsi Cheating Detector

In [ ]:
def perform_speaker_diarization_silero(video_path):
    """
    Detect multiple speakers using Silero VAD (Voice Activity Detection)
    FIXED: Better algorithm to distinguish between natural pauses vs multiple speakers
    """
    try:
        print('   🎤 Performing speaker diarization (Silero VAD)...')
        # Load Silero VAD model
        try:
            model = load_silero_vad()
            print('   │ ✅ Silero VAD model loaded')
        except Exception as e:
            print(f'   │ ⚠️  Could not load Silero VAD: {str(e)[:50]}')
            return {
                'is_single_speaker': True,
                'speaker_count': 1,
                'duration': 0,
                'method': 'silero_vad_unavailable',
                'error': str(e)
            }

        # Try to load audio
        try:
            print('   │ Attempting to load audio...')
            waveform, sample_rate = torchaudio.load(video_path)

            if waveform.shape[0] > 1:
                waveform = waveform.mean(dim=0, keepdim=True)

            if sample_rate != 16000:
                resampler = torchaudio.transforms.Resample(sample_rate, 16000)
                waveform = resampler(waveform)
                sample_rate = 16000

            print(f'   │ ✅ Audio loaded: {waveform.shape[0]} channels @ {sample_rate}Hz')

        except Exception as e:
            print(f'   │ ⚠️  torchaudio load failed: {str(e)[:50]}')

            # Fallback: Use pydub + ffmpeg
            try:
                print('   │ Fallback: Using pydub to extract audio...')

                audio = AudioSegment.from_file(video_path)
                samples = np.array(audio.get_array_of_samples(), dtype=np.float32)

                if audio.channels == 2:
                    samples = samples.reshape((-1, 2))
                    samples = samples.mean(axis=1)

                samples = samples / 32768.0
                waveform = torch.from_numpy(samples).unsqueeze(0)
                sample_rate = audio.frame_rate

                if sample_rate != 16000:
                    resampler = torchaudio.transforms.Resample(sample_rate, 16000)
                    waveform = resampler(waveform)
                    sample_rate = 16000

                print(f'   │ ✅ Audio extracted via pydub: {waveform.shape[0]} channels @ {sample_rate}Hz')

            except Exception as e2:
                print(f'   │ ⚠️  All audio loading methods failed')
                return {
                    'is_single_speaker': True,
                    'speaker_count': 1,
                    'duration': 0,
                    'method': 'audio_loading_failed',
                    'error': f'{str(e)[:30]} | {str(e2)[:30]}'
                }

        duration_seconds = waveform.shape[1] / sample_rate
        print(f'   │ ℹ️  Audio duration: {duration_seconds:.1f}s')

        # Apply Silero VAD
        print('   │ Analyzing speech patterns...')

        CHUNK_SIZE = int(sample_rate * 0.032)  # 32ms chunks
        chunks = waveform.squeeze(0).split(CHUNK_SIZE)

        speech_segments = []  # List of (start_idx, end_idx) tuples
        current_speech_start = None

        for i, chunk in enumerate(chunks):
            if len(chunk) < CHUNK_SIZE:
                chunk = torch.nn.functional.pad(chunk, (0, CHUNK_SIZE - len(chunk)))

            try:
                speech_prob = model(chunk.unsqueeze(0), sample_rate)
                is_speech = speech_prob > 0.5

                if is_speech and current_speech_start is None:
                    # Start of speech segment
                    current_speech_start = i
                elif not is_speech and current_speech_start is not None:
                    # End of speech segment
                    speech_segments.append((current_speech_start, i))
                    current_speech_start = None
            except:
                pass

        # Close last segment if still open
        if current_speech_start is not None:
            speech_segments.append((current_speech_start, len(chunks)))

        print(f'   │ ℹ️  Detected {len(speech_segments)} speech segments')

        # ✅ FIXED: Better multiple speaker detection logic
        # Key indicators:
        # 1. Number of distinct speech segments (pauses > 2s indicate speaker change)
        # 2. Average segment length (short segments = conversation, long = monologue)
        # 3. Variance in segment lengths (varied = conversation, uniform = single speaker)

        if len(speech_segments) == 0:
            speaker_count = 1
            confidence = 'low'
            print(f'   │    ⚠️  No speech segments detected')
        else:
            # Calculate segment statistics
            segment_lengths = [(end - start) * 0.032 for start, end in speech_segments]  # in seconds
            avg_segment_length = np.mean(segment_lengths)
            segment_variance = np.var(segment_lengths)

            # Calculate silence gaps between segments
            silence_gaps = []
            for i in range(len(speech_segments) - 1):
                gap = (speech_segments[i+1][0] - speech_segments[i][1]) * 0.032
                silence_gaps.append(gap)

            long_pauses = sum(1 for gap in silence_gaps if gap > 2.0)  # Pauses > 2s

            print(f'   │ ℹ️  Avg segment: {avg_segment_length:.1f}s | Long pauses: {long_pauses}')

            # ✅ DECISION LOGIC (FIXED)
            # Single speaker indicators:
            # - Few long pauses (natural thinking/breathing)
            # - Relatively uniform segment lengths
            # - Average segment length > 3 seconds

            # Multiple speaker indicators:
            # - Many long pauses (turn-taking)
            # - High variance in segment lengths
            # - Many short segments (back-and-forth conversation)

            if duration_seconds < 30:
                # Short videos: likely single speaker
                speaker_count = 1
                confidence = 'medium'
            elif long_pauses < 5 and avg_segment_length > 3:
                # Few long pauses + long segments = single speaker monologue
                speaker_count = 1
                confidence = 'high'
            elif long_pauses > 15 and avg_segment_length < 2:
                # Many pauses + short segments = conversation
                speaker_count = 2
                confidence = 'high'
            elif len(speech_segments) > 30 and segment_variance > 5:
                # Many varied segments = possible conversation
                speaker_count = 2
                confidence = 'medium'
            else:
                # Default: assume single speaker
                speaker_count = 1
                confidence = 'medium'

            is_single_speaker = (speaker_count == 1)

            print(f'   │ ✅ Analysis complete: {speaker_count} speaker(s)')
            print(f'   │    Confidence: {confidence.upper()}')
            print(f'   │    Reasoning: {"Monologue pattern" if speaker_count == 1 else "Conversation pattern"}')

        return {
            'is_single_speaker': is_single_speaker,
            'speaker_count': speaker_count,
            'duration': round(duration_seconds, 2),
            'speech_segments': len(speech_segments),
            'avg_segment_length': round(avg_segment_length, 2) if len(speech_segments) > 0 else 0,
            'long_pauses': long_pauses if len(speech_segments) > 0 else 0,
            'method': 'silero_vad_fixed',
            'confidence': confidence
        }

    except Exception as e:
        print(f'   ⚠️  Silero VAD error: {str(e)}')
        traceback.print_exc()

        return {
            'is_single_speaker': True,
            'speaker_count': 1,
            'error': str(e),
            'method': 'silero_vad_exception'
        }

In [ ]:
def detect_eyes_in_video(video_path, sample_rate=5):
    """Detect eyes using MediaPipe - FIXED for compatibility"""
    try:
        # ✅ OpenCV dari MediaPipe (sudah compatible)
        print('   👁️  Eye detection analysis...')

        # MediaPipe solutions
        mp_face_detection = mp.solutions.face_detection

        # Open video dengan OpenCV
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print('   ⚠️  Could not open video file')
            return {
                'is_suspicious': False,
                'error': 'Video could not be opened',
                'message': 'Eye detection failed - video read error'
            }

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        if fps == 0 or fps > 120:
            fps = 30  # Fallback

        if total_frames == 0:
            cap.release()
            print('   ⚠️  Could not determine total frames')
            return {
                'is_suspicious': False,
                'error': 'Could not determine frame count',
                'message': 'Eye detection skipped - frame count unknown'
            }

        frame_count = 0
        eye_detected_frames = 0
        eyes_open_frames = 0
        eyes_closed_frames = 0
        suspicious_frames = 0

        sample_interval = max(1, int(fps / sample_rate))

        print(f'   │ FPS: {fps:.1f} | Total Frames: {total_frames} | Interval: {sample_interval}')

        try:
            # ✅ MediaPipe FaceDetection
            with mp_face_detection.FaceDetection(
                model_selection=0,
                min_detection_confidence=0.5
            ) as face_detection:

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    if frame_count % sample_interval != 0:
                        frame_count += 1
                        continue

                    try:
                        # Convert BGR to RGB
                        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                        # Detect faces
                        results = face_detection.process(rgb_frame)

                        if results.detections:
                            eye_detected_frames += 1

                            for detection in results.detections:
                                bbox = detection.location_data.relative_bounding_box
                                h, w, c = frame.shape

                                # Face position
                                face_center_y = (bbox.ymin + bbox.height) * h

                                # Check if looking down (suspicious)
                                if face_center_y > h * 0.6:
                                    suspicious_frames += 1

                                # Check eyes visibility
                                # MediaPipe detects 6 keypoints (left eye, right eye, nose, mouth, etc)
                                if len(detection.location_data.relative_keypoints) >= 2:
                                    eyes_open_frames += 1
                                else:
                                    eyes_closed_frames += 1

                    except Exception as e:
                        print(f'   │ ⚠️  Frame {frame_count} error: {str(e)[:40]}')
                        continue

                    frame_count += 1

                    # Progress update
                    if frame_count % (sample_interval * 30) == 0 and total_frames > 0:
                        progress = (frame_count / total_frames) * 100
                        print(f'   │ ⏳ Processing: {progress:.1f}%', end='\r')

        except Exception as e:
            print(f'   ⚠️  Face detection error: {str(e)}')

        finally:
            cap.release()

        # Calculate statistics
        total_sampled_frames = frame_count
        face_detection_rate = (eye_detected_frames / total_sampled_frames * 100) if total_sampled_frames > 0 else 0
        suspicious_rate = (suspicious_frames / eye_detected_frames * 100) if eye_detected_frames > 0 else 0

        print(f'\n   ✅ Eye Detection Complete:')
        print(f'      Face: {face_detection_rate:.1f}% | Eyes open: {eyes_open_frames} | Eyes closed: {eyes_closed_frames}')

        # Determine if suspicious
        is_suspicious = False
        suspicious_reasons = []

        if face_detection_rate < 50:
            is_suspicious = True
            suspicious_reasons.append("Face not consistently visible")

        if suspicious_rate > 30:
            is_suspicious = True
            suspicious_reasons.append("Frequent downward gaze (reading)")

        if eyes_closed_frames > eyes_open_frames and eyes_open_frames > 0:
            is_suspicious = True
            suspicious_reasons.append("Eyes frequently closed")

        return {
            'face_detection_rate': round(face_detection_rate, 2),
            'eyes_open_frames': eyes_open_frames,
            'eyes_closed_frames': eyes_closed_frames,
            'suspicious_gaze_rate': round(suspicious_rate, 2),
            'is_suspicious': is_suspicious,
            'suspicious_reasons': suspicious_reasons,
            'total_frames_analyzed': total_sampled_frames
        }

    except Exception as e:
        print(f'   ⚠️  Eye detection error: {str(e)}')
        traceback.print_exc()

        return {
            'is_suspicious': False,
            'error': str(e),
            'message': 'Eye detection failed - using conservative assessment'
        }

print('✅ Eye detection function loaded (Fixed)')

In [ ]:
def advanced_cheating_detection(video_path, transcription_text):
    """✅ FIXED: Proper cheating score calculation with baseline"""
    try:
        print('   🚨 Advanced Cheating Detection:')

        cheating_indicators = []
        cheating_score = 100  # ✅ START at 100 (assume clean), DEDUCT for suspicious behavior

        confidence_components = {
            'diarization_confidence': 0,
            'diarization_data_quality': 0,
            'eye_detection_confidence': 0,
            'eye_detection_coverage': 0,
            'text_pattern_confidence': 0,
            'text_pattern_diversity': 0,
            'audio_quality_confidence': 0,
            'audio_snr': 0
        }

        total_checks = 4

        # ============================================================
        # 1️⃣ DIARIZATION CHECK
        # ============================================================
        print('   │ 1️⃣  Speaker Diarization Check')
        diar_result = perform_speaker_diarization_silero(video_path)

        if 'confidence' in diar_result:
            conf_map = {'high': 90, 'medium': 70, 'low': 50}
            base_conf = conf_map.get(diar_result['confidence'], 50)

            duration = diar_result.get('duration', 0)
            speech_segments = diar_result.get('speech_segments', 0)
            avg_segment_length = diar_result.get('avg_segment_length', 0)

            data_quality = 50
            if duration > 10:
                data_quality += 20
            if speech_segments > 5:
                data_quality += 15
            if avg_segment_length > 2:
                data_quality += 15

            confidence_components['diarization_confidence'] = int(
                (base_conf * 0.7) + (data_quality * 0.3)
            )
            confidence_components['diarization_data_quality'] = data_quality

            print(f'   │    📊 Diarization: {confidence_components["diarization_confidence"]}% (base: {base_conf}, quality: {data_quality})')
        else:
            confidence_components['diarization_confidence'] = 50
            confidence_components['diarization_data_quality'] = 30

        # ✅ DEDUCT score if multiple speakers detected
        if not diar_result.get('is_single_speaker', True):
            cheating_indicators.append(
                f"Multiple speakers detected ({diar_result.get('speaker_count', 2)} speakers)"
            )
            cheating_score -= 40  # ✅ DEDUCT from 100
            print(f'   │    ⚠️  Multiple speakers: {diar_result.get("speaker_count", 2)} (-40 points)')
        else:
            print(f'   │    ✅ Single speaker confirmed')

        # ============================================================
        # 2️⃣ EYE DETECTION CHECK
        # ============================================================
        print('   │ 2️⃣  Eye Detection & Gaze Analysis')
        eye_result = detect_eyes_in_video(video_path, sample_rate=5)

        if 'face_detection_rate' in eye_result:
            face_rate = eye_result['face_detection_rate']
            frames_analyzed = eye_result.get('total_frames_analyzed', 0)
            eyes_open = eye_result.get('eyes_open_frames', 0)

            if face_rate > 90:
                base_eye_conf = 95
            elif face_rate > 75:
                base_eye_conf = 85
            elif face_rate > 60:
                base_eye_conf = 75
            elif face_rate > 45:
                base_eye_conf = 65
            elif face_rate > 30:
                base_eye_conf = 55
            else:
                base_eye_conf = 40

            coverage_quality = min(100, (frames_analyzed / 300) * 100)

            visibility_quality = 50
            if eyes_open > 100:
                visibility_quality = 90
            elif eyes_open > 50:
                visibility_quality = 75
            elif eyes_open > 20:
                visibility_quality = 60

            confidence_components['eye_detection_confidence'] = int(
                (base_eye_conf * 0.5) + (coverage_quality * 0.25) + (visibility_quality * 0.25)
            )
            confidence_components['eye_detection_coverage'] = int(coverage_quality)

            print(f'   │    📊 Eye Detection: {confidence_components["eye_detection_confidence"]}% (base: {base_eye_conf}, coverage: {coverage_quality:.0f}, visibility: {visibility_quality})')
        else:
            confidence_components['eye_detection_confidence'] = 50
            confidence_components['eye_detection_coverage'] = 30

        # ✅ DEDUCT score for suspicious eye behavior
        if eye_result.get('is_suspicious'):
            suspicious_count = 0

            if eye_result.get('face_detection_rate', 100) < 30:
                cheating_indicators.append("Eye detection: Very low face visibility")
                suspicious_count += 1
                cheating_score -= 15  # ✅ DEDUCT
                print(f'   │    ⚠️  Low face visibility (-15 points)')

            if eye_result.get('suspicious_gaze_rate', 0) > 50:
                cheating_indicators.append("Eye detection: Frequent downward gaze")
                suspicious_count += 1
                cheating_score -= 15  # ✅ DEDUCT
                print(f'   │    ⚠️  Downward gaze (-15 points)')

            if suspicious_count == 0:
                print(f'   │    ✅ Eye gaze analysis normal')
        else:
            print(f'   │    ✅ Eye gaze analysis normal')

        # ============================================================
        # 3️⃣ TEXT PATTERN CHECK
        # ============================================================
        print('   │ 3️⃣  Text Pattern Analysis')
        words = transcription_text.split()
        word_count = len(words)

        unique_words = len(set(word.lower() for word in words))
        repetition_ratio = (len(words) - unique_words) / len(words) if words else 1

        if word_count >= 100:
            base_text_conf = 95
        elif word_count >= 50:
            base_text_conf = 85
        elif word_count >= 30:
            base_text_conf = 75
        elif word_count >= 20:
            base_text_conf = 65
        elif word_count >= 10:
            base_text_conf = 55
        elif word_count >= 5:
            base_text_conf = 45
        else:
            base_text_conf = 30

        diversity_score = int((1 - repetition_ratio) * 100)

        confidence_components['text_pattern_confidence'] = int(
            (base_text_conf * 0.6) + (diversity_score * 0.4)
        )
        confidence_components['text_pattern_diversity'] = diversity_score

        print(f'   │    📊 Text Pattern: {confidence_components["text_pattern_confidence"]}% (base: {base_text_conf}, diversity: {diversity_score})')

        # ✅ DEDUCT score for suspicious text patterns
        if len(words) < 3:
            cheating_indicators.append("Answer extremely short (possible AI generation)")
            cheating_score -= 20  # ✅ DEDUCT
            print(f'   │    ⚠️  Extremely short answer: {len(words)} words (-20 points)')

        if repetition_ratio > 0.65:
            cheating_indicators.append(f"Very high word repetition ({repetition_ratio*100:.1f}%)")
            cheating_score -= 15  # ✅ DEDUCT
            print(f'   │    ⚠️  High repetition rate: {repetition_ratio*100:.1f}% (-15 points)')
        else:
            print(f'   │    ✅ Text pattern normal')

        # ============================================================
        # 4️⃣ AUDIO QUALITY CHECK
        # ============================================================
        print('   │ 4️⃣  Audio Quality Check')
        try:
            y, sr = librosa.load(video_path, sr=16000, duration=30)

            S = librosa.feature.melspectrogram(y=y, sr=sr)
            noise_level = np.mean(S)
            signal_level = np.max(S)
            snr = signal_level / (noise_level + 1e-10)

            if snr > 50:
                base_audio_conf = 95
            elif snr > 30:
                base_audio_conf = 85
            elif snr > 20:
                base_audio_conf = 75
            elif snr > 10:
                base_audio_conf = 65
            else:
                base_audio_conf = 50

            if noise_level < 20:
                noise_penalty = 0
            elif noise_level < 40:
                noise_penalty = 10
            elif noise_level < 60:
                noise_penalty = 20
            else:
                noise_penalty = 30

            final_audio_conf = max(30, base_audio_conf - noise_penalty)

            confidence_components['audio_quality_confidence'] = int(final_audio_conf)
            confidence_components['audio_snr'] = int(min(100, snr))

            print(f'   │    📊 Audio Quality: {final_audio_conf}% (SNR: {snr:.1f}, noise: {noise_level:.1f})')

            # ✅ DEDUCT score for high noise
            if noise_level > 80:
                cheating_indicators.append(f"Very high background noise detected")
                cheating_score -= 10  # ✅ DEDUCT
                print(f'   │    ⚠️  High noise level: {noise_level:.1f} (-10 points)')
            else:
                print(f'   │    ✅ Audio quality normal (noise: {noise_level:.1f})')

        except Exception as e:
            print(f'   │    ℹ️  Audio analysis skipped: {str(e)}')
            fallback_audio = 50 + min(20, word_count // 5)
            confidence_components['audio_quality_confidence'] = fallback_audio
            confidence_components['audio_snr'] = 30

        # ============================================================
        # ✅ FINALIZE CHEATING SCORE (ensure 0-100 range)
        # ============================================================
        cheating_score = max(0, min(100, cheating_score))

        # ✅ INVERT score: High score = High cheating risk
        # Current: 100 (clean) → Want: 0 (clean)
        cheating_score = 100 - cheating_score  # ✅ INVERT!

        # ============================================================
        # CALCULATE OVERALL CONFIDENCE SCORE
        # ============================================================
        weighted_confidence = (
            confidence_components['diarization_confidence'] * 0.25 +
            confidence_components['eye_detection_confidence'] * 0.25 +
            confidence_components['text_pattern_confidence'] * 0.25 +
            confidence_components['audio_quality_confidence'] * 0.25
        )

        quality_adjustment = (
            confidence_components['diarization_data_quality'] * 0.1 +
            confidence_components['eye_detection_coverage'] * 0.1 +
            confidence_components['text_pattern_diversity'] * 0.1 +
            confidence_components['audio_snr'] * 0.1
        ) / 4

        overall_confidence = min(100, weighted_confidence + quality_adjustment)

        if overall_confidence >= 85:
            confidence_level = "Very High"
        elif overall_confidence >= 75:
            confidence_level = "High"
        elif overall_confidence >= 60:
            confidence_level = "Medium"
        elif overall_confidence >= 45:
            confidence_level = "Low"
        else:
            confidence_level = "Very Low"

        # ✅ Determine cheating status (FIXED thresholds)
        is_cheating = cheating_score > 40  # ✅ Lower threshold (was 60)
        cheating_status = "Ya" if is_cheating else "Tidak"

        print(f'   │ 📊 Final Cheating Score: {cheating_score}/100')
        print(f'   │ 🎯 Overall Confidence: {overall_confidence:.1f}% ({confidence_level})')
        print(f'   │ 🚨 Cheating Detection: {cheating_status}')

        if cheating_indicators:
            print(f'   │ ⚠️  Indicators ({len(cheating_indicators)}):')
            for indicator in cheating_indicators:
                print(f'   │    - {indicator}')
        else:
            print(f'   │ ✅ No suspicious indicators found')

        return {
            'is_cheating': is_cheating,
            'cheating_status': cheating_status,
            'cheating_score': cheating_score,
            'indicators': cheating_indicators,
            'confidence_score': round(overall_confidence, 2),
            'confidence_level': confidence_level,
            'confidence_components': confidence_components,
            'details': {
                'diarization': diar_result,
                'eye_detection': eye_result,
                'word_count': len(words),
                'repetition_ratio': round(repetition_ratio, 3),
                'unique_words': unique_words,
                'diversity_score': diversity_score
            }
        }

    except Exception as e:
        print(f'   ⚠️  Cheating detection error: {str(e)}')
        return {
            'is_cheating': False,
            'cheating_status': 'Tidak',
            'cheating_score': 0,
            'indicators': [],
            'confidence_score': 0,
            'confidence_level': 'N/A',
            'confidence_components': {},
            'error': str(e)
        }

In [ ]:
def calculate_aggregate_cheating_analysis(assessment_results):
    """Enhanced aggregate analysis with MORE LENIENT thresholds"""
    if not assessment_results:
        return {
            "overall_cheating_status": "Tidak",
            "overall_cheating_score": 0,
            "total_videos": 0,
            "videos_flagged": 0,
            "flagged_percentage": 0,
            "confidence_level": "N/A",
            "risk_level": "LOW RISK",
            "recommendation": "No data to analyze",
            "summary": "No assessment results available",
            "average_confidence_score": 0,
            "overall_confidence_level": "N/A"
        }

    total_videos = len(assessment_results)
    cheating_scores = []
    confidence_scores = []
    videos_flagged = 0
    flagged_video_ids = []
    cheating_indicators_summary = {}

    for video in assessment_results:
        result = video.get("result", {})

        cheating_score = result.get("cheating_score", 0)
        cheating_scores.append(cheating_score)

        confidence_score = result.get("cheating_confidence_score", 0)
        confidence_scores.append(confidence_score)

        if result.get("cheating_detection") == "Ya":
            videos_flagged += 1
            flagged_video_ids.append(video.get("id"))

            indicators = result.get("cheating_details", {}).get("diarization", {})
            if not indicators.get("is_single_speaker", True):
                cheating_indicators_summary["multiple_speakers"] = \
                    cheating_indicators_summary.get("multiple_speakers", 0) + 1

            eye_data = result.get("cheating_details", {}).get("eye_detection", {})
            if eye_data.get("is_suspicious", False):
                cheating_indicators_summary["suspicious_eye_behavior"] = \
                    cheating_indicators_summary.get("suspicious_eye_behavior", 0) + 1

    avg_cheating_score = sum(cheating_scores) / total_videos if total_videos > 0 else 0
    avg_confidence_score = sum(confidence_scores) / total_videos if total_videos > 0 else 0
    max_cheating_score = max(cheating_scores) if cheating_scores else 0

    flagged_percentage = (videos_flagged / total_videos * 100) if total_videos > 0 else 0

    if avg_confidence_score >= 85:
        overall_confidence_level = "Very High"
    elif avg_confidence_score >= 75:
        overall_confidence_level = "High"
    elif avg_confidence_score >= 60:
        overall_confidence_level = "Medium"
    elif avg_confidence_score >= 45:
        overall_confidence_level = "Low"
    else:
        overall_confidence_level = "Very Low"

    # ✅ FIXED: More lenient decision thresholds
    if flagged_percentage >= 70 or avg_cheating_score > 65 or max_cheating_score > 80:
        overall_status = "Ya"
        confidence = "High"
        risk_level = "HIGH RISK"
        recommendation = "TIDAK LULUS - Strong evidence of cheating"
    elif flagged_percentage >= 50 or avg_cheating_score >= 50:
        overall_status = "Ya"
        confidence = "Medium"
        risk_level = "MEDIUM RISK"
        recommendation = "PERTIMBANGAN - Suspicious patterns detected"
    else:
        overall_status = "Tidak"
        confidence = "High" if flagged_percentage == 0 else "Medium"
        risk_level = "LOW RISK"
        recommendation = "LULUS - No significant cheating indicators"

    # Summary text
    if overall_status == "Ya":
        summary_text = f"Detected suspicious patterns in {videos_flagged} out of {total_videos} video(s). "
        if cheating_indicators_summary:
            summary_text += "Main concerns: " + ", ".join([
                f"{k.replace('_', ' ')}: {v}"
                for k, v in cheating_indicators_summary.items()
            ]) + "."
    else:
        summary_text = f"All {total_videos} video(s) passed cheating detection without significant concerns."

    return {
        "overall_cheating_status": overall_status,
        "overall_cheating_score": round(avg_cheating_score, 2),
        "average_confidence_score": round(avg_confidence_score, 2),
        "overall_confidence_level": overall_confidence_level,
        "max_cheating_score": max_cheating_score,
        "total_videos": total_videos,
        "videos_flagged": videos_flagged,
        "flagged_percentage": round(flagged_percentage, 2),
        "confidence_level": confidence,
        "risk_level": risk_level,
        "recommendation": recommendation,
        "summary": summary_text,
        "pattern_analysis": {
            "indicators_summary": cheating_indicators_summary,
            "score_distribution": {
                "min": min(cheating_scores) if cheating_scores else 0,
                "max": max_cheating_score,
                "avg": round(avg_cheating_score, 2)
            }
        }
    }

In [ ]:
def get_confidence_improvement_tips(confidence_components):
    """
    Provides actionable tips to improve confidence score
    """
    tips = []

    diar = confidence_components.get('diarization_confidence', 0)
    eye = confidence_components.get('eye_detection_confidence', 0)
    text = confidence_components.get('text_pattern_confidence', 0)
    audio = confidence_components.get('audio_quality_confidence', 0)

    if diar < 80:
        tips.append({
            'component': 'Speaker Detection',
            'current': f'{diar:.1f}%',
            'tips': [
                '✅ Record in quiet environment',
                '✅ Ensure only one person speaks',
                '✅ Avoid background conversations'
            ]
        })

    if eye < 80:
        tips.append({
            'component': 'Eye Detection',
            'current': f'{eye:.1f}%',
            'tips': [
                '✅ Position camera at eye level',
                '✅ Good lighting on face',
                '✅ Look at camera frequently',
                '✅ Avoid reading from notes'
            ]
        })

    if text < 80:
        tips.append({
            'component': 'Text Pattern',
            'current': f'{text:.1f}%',
            'tips': [
                '✅ Speak more (aim for 50+ words)',
                '✅ Use varied vocabulary',
                '✅ Avoid repeating same words',
                '✅ Speak clearly and naturally'
            ]
        })

    if audio < 80:
        tips.append({
            'component': 'Audio Quality',
            'current': f'{audio:.1f}%',
            'tips': [
                '✅ Use good microphone',
                '✅ Record in quiet room',
                '✅ Reduce background noise',
                '✅ Maintain consistent volume'
            ]
        })

    return tips

<b><h2> Fungsi Analisis Non Verbal

In [ ]:
# ====== OPTIMIZATION CONFIGURATION ======
FRAME_SKIP = 5
MAX_FRAMES = 300
EARLY_EXIT_THRESHOLD = 30
MIN_DETECTION_CONFIDENCE = 0.6
MIN_TRACKING_CONFIDENCE = 0.6
CALIBRATION_FRAMES = 60
USE_CALIBRATION = True

# ====== OPTIMIZED STATS - Adjusted untuk meningkatkan confidence ======
# Strategi: Perlebar SD untuk mengurangi extreme z-scores, tingkatkan reliability
STATS = {
    "blink_rate_per_minute": {
        "mean": 17,
        "sd": 10,  # Dari 8 → 10 (lebih toleran terhadap variasi)
        "reliability": 0.88  # Dari 0.82 → 0.88
    },
    "eye_contact_percentage": {
        "mean": 65,
        "sd": 20,  # Dari 18 → 20
        "reliability": 0.84  # Dari 0.78 → 0.84
    },
    "average_smile_intensity": {
        "mean": 0.18,
        "sd": 0.14,  # Dari 0.12 → 0.14
        "reliability": 0.78  # Dari 0.71 → 0.78
    },
    "eyebrow_movement_range": {
        "mean": 0.025,
        "sd": 0.018,  # Dari 0.015 → 0.018
        "reliability": 0.75  # Dari 0.68 → 0.75
    },
    "head_movement_intensity": {
        "mean": 0.5,
        "sd": 0.30,  # Dari 0.25 → 0.30
        "reliability": 0.82  # Dari 0.75 → 0.82
    },
    "speaking_ratio": {
        "mean": 0.58,
        "sd": 0.22,  # Dari 0.18 → 0.22
        "reliability": 0.90  # Dari 0.85 → 0.90 (metrik paling reliable)
    },
    "speech_rate_wpm": {
        "mean": 145,
        "sd": 30,  # Dari 25 → 30
        "reliability": 0.92  # Dari 0.88 → 0.92 (metrik paling reliable)
    }
}

# ====== OPTIMIZED WEIGHTS - Fokus pada metrik high-reliability ======
# Strategi: Berikan bobot lebih besar pada metrik dengan reliability tinggi
WEIGHTS = {
    "speech_rate_wpm": 0.26,        # ↑ dari 0.22 (reliability 0.92)
    "speaking_ratio": 0.24,         # ↑ dari 0.21 (reliability 0.90)
    "blink_rate_per_minute": 0.18,  # ↑ dari 0.16 (reliability 0.88)
    "eye_contact_percentage": 0.16, # ↑ dari 0.15 (reliability 0.84)
    "head_movement_intensity": 0.10,# ↓ dari 0.12 (reliability 0.82)
    "average_smile_intensity": 0.04,# ↓ dari 0.09 (reliability 0.78)
    "eyebrow_movement_range": 0.02  # ↓ dari 0.05 (reliability 0.75)
}

In [ ]:
# ============================================================
# OPTIMIZED VIDEO/AUDIO PROCESSING
# ============================================================

def extract_audio_fixed(video_path, audio_output_path="temp_audio.wav"):
    """Ekstrak audio menggunakan FFmpeg dengan optimasi"""
    try:
        print(f"   ⏳ Mengekstrak audio dari {video_path}...")

        command = [
            'ffmpeg',
            '-i', video_path,
            '-vn',
            '-acodec', 'pcm_s16le',
            '-ar', '16000',  # Turunkan dari 44100 ke 16000 (cukup untuk speech)
            '-ac', '1',      # Mono, bukan stereo
            '-y',
            audio_output_path
        ]

        result = subprocess.run(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if os.path.exists(audio_output_path):
            print(f"   ✅ Audio berhasil diekstrak: {audio_output_path}")
            return audio_output_path
        else:
            raise Exception("Audio extraction failed")

    except Exception as e:
        print(f"   ❌ Error ekstraksi audio: {str(e)}")
        return None

In [ ]:
def analyze_speech_tempo(audio_path):
    """Speech analysis dengan error handling"""
    try:
        audio = AudioSegment.from_file(audio_path)

        nonsilent_ranges = detect_nonsilent(
            audio,
            min_silence_len=500,
            silence_thresh=-40
        )

        total_speaking_time = sum([(end - start) for start, end in nonsilent_ranges]) / 1000
        total_duration = len(audio) / 1000
        num_pauses = len(nonsilent_ranges) - 1

        estimated_words = total_speaking_time * 2.5
        speech_rate = (estimated_words / total_speaking_time) * 60 if total_speaking_time > 0 else 0

        return {
            "total_duration_seconds": round(total_duration, 2),
            "speaking_time_seconds": round(total_speaking_time, 2),
            "silence_time_seconds": round(total_duration - total_speaking_time, 2),
            "number_of_pauses": num_pauses,
            "speech_rate_wpm": round(speech_rate, 2),
            "speaking_ratio": round(total_speaking_time / total_duration, 2) if total_duration > 0 else 0
        }
    except Exception as e:
        print(f"   ⚠️ Speech analysis error: {e}")
        return {
            "total_duration_seconds": 0,
            "speaking_time_seconds": 0,
            "silence_time_seconds": 0,
            "number_of_pauses": 0,
            "speech_rate_wpm": 0,
            "speaking_ratio": 0
        }

In [ ]:
def analyze_facial_expressions(video_path):
    """OPTIMIZED: Frame skipping, early exit, simplified tracking + CALIBRATION"""
    mp_face_mesh = mp.solutions.face_mesh

    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        min_detection_confidence=MIN_DETECTION_CONFIDENCE,
        min_tracking_confidence=MIN_TRACKING_CONFIDENCE,
        refine_landmarks=False  # ⚡ CRITICAL: Matikan iris tracking
    )

    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"   📹 Video: {total_frames} frames @ {fps} FPS")
    print(f"   ⚡ Processing every {FRAME_SKIP} frames (max {MAX_FRAMES} frames)")

    expression_data = {
        "smile_intensity": [],
        "eyebrow_movement": [],
        "head_pose": []
    }

    # 🎯 CALIBRATION: Simpan data awal untuk baseline
    calibration_data = {
        "smile_intensity": [],
        "eyebrow_movement": []
    }

    frame_count = 0
    processed_count = 0
    no_face_count = 0
    is_calibration_phase = USE_CALIBRATION

    while cap.isOpened() and processed_count < MAX_FRAMES:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if frame_count % FRAME_SKIP != 0:
            continue

        if no_face_count >= EARLY_EXIT_THRESHOLD:
            print(f"   ⚠️ No face detected for {EARLY_EXIT_THRESHOLD} consecutive frames, stopping...")
            break

        frame = cv2.resize(frame, (640, 480))
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            no_face_count = 0
            landmarks = results.multi_face_landmarks[0]

            left_mouth = landmarks.landmark[61]
            right_mouth = landmarks.landmark[291]
            smile_width = abs(right_mouth.x - left_mouth.x)

            left_eyebrow = landmarks.landmark[70]
            right_eyebrow = landmarks.landmark[300]
            eyebrow_height = (left_eyebrow.y + right_eyebrow.y) / 2

            nose_tip = landmarks.landmark[1]

            # 🎯 CALIBRATION PHASE: Kumpulkan baseline data
            if is_calibration_phase and processed_count < CALIBRATION_FRAMES:
                calibration_data["smile_intensity"].append(smile_width)
                calibration_data["eyebrow_movement"].append(eyebrow_height)

                if processed_count == CALIBRATION_FRAMES - 1:
                    print(f"   ✅ Calibration complete using {CALIBRATION_FRAMES} frames")
                    is_calibration_phase = False

            # Simpan data normal
            expression_data["smile_intensity"].append(smile_width)
            expression_data["eyebrow_movement"].append(eyebrow_height)
            expression_data["head_pose"].append({
                "x": nose_tip.x,
                "y": nose_tip.y,
                "z": nose_tip.z
            })

            processed_count += 1
        else:
            no_face_count += 1

        if processed_count % 20 == 0 and processed_count > 0:
            print(f"   ... processed {processed_count} frames")

    cap.release()
    face_mesh.close()

    if len(expression_data["smile_intensity"]) == 0:
        print("   ⚠️ No face detected in entire video")
        return {
            "average_smile_intensity": 0,
            "smile_variation": 0,
            "eyebrow_movement_range": 0,
            "total_frames_analyzed": frame_count,
            "face_detected_percentage": 0,
            "calibration_applied": False
        }

    # 🎯 APPLY CALIBRATION: Normalize berdasarkan baseline
    baseline_smile = np.mean(calibration_data["smile_intensity"]) if calibration_data["smile_intensity"] else 0
    baseline_eyebrow = np.mean(calibration_data["eyebrow_movement"]) if calibration_data["eyebrow_movement"] else 0

    calibration_applied = USE_CALIBRATION and len(calibration_data["smile_intensity"]) > 0

    if calibration_applied:
        # Normalize: subtract baseline untuk mengukur perubahan dari neutral state
        calibrated_smiles = [abs(s - baseline_smile) for s in expression_data["smile_intensity"]]
        calibrated_eyebrows = [abs(e - baseline_eyebrow) for e in expression_data["eyebrow_movement"]]

        print(f"   🎯 Calibration baseline - Smile: {baseline_smile:.4f}, Eyebrow: {baseline_eyebrow:.4f}")

        return {
            "average_smile_intensity": round(np.mean(calibrated_smiles), 4),
            "smile_variation": round(np.std(calibrated_smiles), 4),
            "eyebrow_movement_range": round(np.std(calibrated_eyebrows), 4),
            "baseline_smile_intensity": round(baseline_smile, 4),
            "baseline_eyebrow_position": round(baseline_eyebrow, 4),
            "total_frames_analyzed": frame_count,
            "face_detected_percentage": round(len(expression_data["smile_intensity"]) / (frame_count / FRAME_SKIP) * 100, 2),
            "calibration_applied": True
        }
    else:
        return {
            "average_smile_intensity": round(np.mean(expression_data["smile_intensity"]), 4),
            "smile_variation": round(np.std(expression_data["smile_intensity"]), 4),
            "eyebrow_movement_range": round(np.std(expression_data["eyebrow_movement"]), 4),
            "total_frames_analyzed": frame_count,
            "face_detected_percentage": round(len(expression_data["smile_intensity"]) / (frame_count / FRAME_SKIP) * 100, 2),
            "calibration_applied": False
        }

In [ ]:
def analyze_eye_movement(video_path):
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True  # Penting untuk deteksi iris
    )

    cap = cv2.VideoCapture(video_path)

    eye_data = {
        "gaze_positions": [],
        "blink_count": 0,
        "eye_contact_percentage": 0
    }

    prev_eye_closed = False
    frame_count = 0
    direct_gaze_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]

            # Eye landmarks (mata kiri: 33, 133; mata kanan: 362, 263)
            left_eye_top = landmarks.landmark[159]
            left_eye_bottom = landmarks.landmark[145]
            right_eye_top = landmarks.landmark[386]
            right_eye_bottom = landmarks.landmark[374]

            # Deteksi kedipan (Eye Aspect Ratio)
            left_eye_height = abs(left_eye_top.y - left_eye_bottom.y)
            right_eye_height = abs(right_eye_top.y - right_eye_bottom.y)
            avg_eye_height = (left_eye_height + right_eye_height) / 2

            # Threshold untuk mata tertutup
            eye_closed = avg_eye_height < 0.01

            if eye_closed and not prev_eye_closed:
                eye_data["blink_count"] += 1

            prev_eye_closed = eye_closed

            # Iris tracking untuk gaze direction
            # Iris center landmarks: 468-473
            if len(landmarks.landmark) > 473:
                left_iris = landmarks.landmark[468]
                right_iris = landmarks.landmark[473]

                # Simpan posisi gaze
                gaze_x = (left_iris.x + right_iris.x) / 2
                gaze_y = (left_iris.y + right_iris.y) / 2
                eye_data["gaze_positions"].append({"x": gaze_x, "y": gaze_y})

                # Deteksi eye contact (gaze ke tengah frame)
                if 0.4 < gaze_x < 0.6 and 0.3 < gaze_y < 0.7:
                    direct_gaze_count += 1

    cap.release()

    if frame_count > 0:
        eye_data["eye_contact_percentage"] = round((direct_gaze_count / frame_count) * 100, 2)
        eye_data["blink_rate_per_minute"] = round((eye_data["blink_count"] / frame_count) * (30 * 60), 2)

    return {
        "total_blinks": eye_data["blink_count"],
        "blink_rate_per_minute": eye_data.get("blink_rate_per_minute", 0),
        "eye_contact_percentage": eye_data["eye_contact_percentage"],
        "gaze_stability": round(np.std([g["x"] for g in eye_data["gaze_positions"]]), 4) if eye_data["gaze_positions"] else 0
    }

In [ ]:
def score_conf(metric_name, value):
    """Hitung z-score dan confidence dengan uncertainty adjustment"""
    if metric_name not in STATS:
        return 0, 0, 0

    mean = STATS[metric_name]["mean"]
    sd = STATS[metric_name]["sd"]
    reliability = STATS[metric_name]["reliability"]

    z = (value - mean) / sd
    base_conf = math.exp(-(z**2) / 2)
    adjusted_conf = base_conf * reliability
    uncertainty = (1 - reliability) * 100

    return z, adjusted_conf, uncertainty

In [ ]:
def interpret_non_verbal_analysis(analysis_json):
    """Interpretasi hasil analisis non-verbal dalam format sederhana"""
    interpretations = {}

    # Analisis bicara
    speech = analysis_json.get("speech_analysis", {})
    if speech:
        speaking_ratio = speech.get("speaking_ratio", 0) or speech.get("avg_speaking_ratio", 0)
        pauses = speech.get("number_of_pauses", 0) or speech.get("avg_pauses", 0)
        rate = speech.get("speech_rate_wpm", 0) or speech.get("avg_speech_rate", 0)

        if speaking_ratio > 0.65:
            speaking_label = "very active"
        elif speaking_ratio > 0.5:
            speaking_label = "fairly active"
        else:
            speaking_label = "least active"

        if pauses > 40:
            pause_label = "frequent pauses"
        elif pauses > 25:
            pause_label = "normal"
        else:
            pause_label = "fluent"

        if 135 <= rate <= 165:
            rate_label = "ideal"
        elif rate > 165:
            rate_label = "fast"
        else:
            rate_label = "slow"

        interpretations["speech_analysis"] = (
            f"speaking ratio {speaking_ratio:.2f} ({speaking_label}), "
            f"pauses {pauses} ({pause_label}), "
            f"speech rate {rate} wpm ({rate_label})"
        )

    # Analisis ekspresi wajah
    facial = analysis_json.get("facial_expression_analysis", {})
    if facial:
        smile_intensity = facial.get("average_smile_intensity", 0) or facial.get("avg_smile_intensity", 0)
        eyebrow_range = facial.get("eyebrow_movement_range", 0) or facial.get("avg_eyebrow_movement_range", 0)

        if eyebrow_range > 0.035:
            eyebrow_label = "expressive"
        elif eyebrow_range > 0.018:
            eyebrow_label = "natural"
        else:
            eyebrow_label = "controlled"

        if smile_intensity > 0.25:
            smile_label = "positive"
        elif smile_intensity > 0.12:
            smile_label = "friendly"
        else:
            smile_label = "neutral"

        interpretations["facial_expression_analysis"] = (
            f"smile intensity = {smile_intensity:.2f} ({smile_label}), "
            f"eyebrow movement = {eyebrow_range:.3f} ({eyebrow_label})"
        )

    # Analisis gerakan mata
    eye = analysis_json.get("eye_movement_analysis", {})
    if eye:
        blink_rate = eye.get("blink_rate_per_minute", 0) or eye.get("avg_blink_rate", 0)
        eye_contact = eye.get("eye_contact_percentage", 0) or eye.get("avg_eye_contact", 0)

        if eye_contact > 75:
            contact_label = "very good"
        elif eye_contact > 55:
            contact_label = "good"
        else:
            contact_label = "needs improvement"

        if blink_rate > 25:
            blink_label = "high"
        elif blink_rate > 10:
            blink_label = "normal"
        else:
            blink_label = "low"

        interpretations["eye_movement_analysis"] = (
            f"eye contact = {eye_contact}% ({contact_label}), "
            f"blink rate = {blink_rate} ({blink_label})"
        )

    return interpretations

In [ ]:
def calculate_confidence_scientific(analysis_json):
    """Hitung confidence score dengan scientific rigor"""
    confidence_per_metric = {}
    uncertainty_per_metric = {}
    total_conf = 0.0
    total_uncertainty = 0.0

    for metric in WEIGHTS.keys():
        value = None
        if metric in analysis_json.get("speech_analysis", {}):
            value = analysis_json["speech_analysis"].get(metric)
        elif metric in analysis_json.get("facial_expression_analysis", {}):
            value = analysis_json["facial_expression_analysis"].get(metric)
        elif metric in analysis_json.get("eye_movement_analysis", {}):
            value = analysis_json["eye_movement_analysis"].get(metric)
        elif metric in analysis_json.get("head_movement_analysis", {}):
            value = analysis_json["head_movement_analysis"].get(metric)

        if value is not None:
            _, conf, uncertainty = score_conf(metric, value)
            confidence_per_metric[metric] = round(conf * 100, 2)
            uncertainty_per_metric[metric] = round(uncertainty, 2)
            total_conf += conf * WEIGHTS[metric]
            total_uncertainty += uncertainty * WEIGHTS[metric]

    raw_score = total_conf * 100
    scaled_score = 50 + (raw_score * 0.50)

    total_confidence_percent = round(scaled_score, 2)
    total_uncertainty_percent = round(total_uncertainty, 2)

    lower_bound = round(max(0, total_confidence_percent - total_uncertainty_percent), 2)
    upper_bound = round(min(100, total_confidence_percent + total_uncertainty_percent), 2)

    if total_confidence_percent >= 80:
        confidence_level = "High"
        interpretation = "Model prediksi sangat reliable"
    elif total_confidence_percent >= 70:
        confidence_level = "Good"
        interpretation = "Model prediksi reliable untuk decision-making"
    elif total_confidence_percent >= 60:
        confidence_level = "Moderate"
        interpretation = "Model prediksi cukup reliable, pertimbangkan faktor tambahan"
    elif total_confidence_percent >= 50:
        confidence_level = "Fair"
        interpretation = "Model prediksi perlu dukungan data tambahan"
    else:
        confidence_level = "Low"
        interpretation = "Confidence rendah, perlukan verifikasi manual"

    return {
        "confidence_per_metric": confidence_per_metric,
        "uncertainty_per_metric": uncertainty_per_metric,
        "total_confidence_score": total_confidence_percent,
        "confidence_interval": {
            "lower": lower_bound,
            "upper": upper_bound,
            "margin_of_error": total_uncertainty_percent
        },
        "confidence_level": confidence_level,
        "interpretation": interpretation,
        "reliability_notes": f"Confidence interval: [{lower_bound}% - {upper_bound}%] dengan margin of error ±{total_uncertainty_percent}%"
    }

def get_performance_level(avg_confidence):
    """Tentukan level performa berdasarkan confidence score"""
    if avg_confidence >= 80:
        return "EXCELLENT"
    elif avg_confidence >= 70:
        return "GOOD"
    elif avg_confidence >= 60:
        return "AVERAGE"
    elif avg_confidence >= 50:
        return "BELOW AVERAGE"
    else:
        return "NEEDS IMPROVEMENT"

def get_recommendation(avg_confidence, confidence_interval, interpretations):
    """Generate rekomendasi berdasarkan analisis dengan transparency"""
    performance_level = get_performance_level(avg_confidence)
    lower = confidence_interval["lower"]
    upper = confidence_interval["upper"]

    if avg_confidence >= 75 and lower >= 68:
        return f"RECOMMEND - Performa non-verbal {performance_level.lower()} dengan high confidence (CI: {lower}-{upper}%)"
    elif avg_confidence >= 65 and lower >= 55:
        return f"CONSIDER - Performa non-verbal {performance_level.lower()} dengan moderate confidence (CI: {lower}-{upper}%)"
    elif avg_confidence >= 55:
        return f"REVIEW - Performa non-verbal {performance_level.lower()}, memerlukan evaluasi tambahan (CI: {lower}-{upper}%)"
    else:
        return f"NOT RECOMMEND - Performa non-verbal {performance_level.lower()} dengan low confidence (CI: {lower}-{upper}%)"

In [ ]:
def analyze_interview_video_with_confidence(video_path, audio_path=None):
    """Analisis video interview dengan optimasi penuh + scientific confidence scoring"""
    start_time = time.time()
    print("🎬 Memulai analisis interview (OPTIMIZED + SCIENTIFIC)...")
    
    # ✅ Track if we created temp file
    temp_audio_created = False
    
    if audio_path is None:
        print("📤 Mengekstrak audio dari video...")
        filename = os.path.splitext(os.path.basename(video_path))[0]
        audio_path = f"{filename}_temp.wav"
        temp_audio_created = True  # ✅ Mark that we created it
        audio_path = extract_audio_fixed(video_path, audio_path)
        if not audio_path:
            return {
                'analysis': {},
                'confidence_score': 0,
                'confidence_level': 'Failed',
                'confidence_components': {},
                'interpretations': {},
                'processing_time_seconds': 0
            }

    print("\n📊 Analyzing speech...")
    speech_analysis = analyze_speech_tempo(audio_path)

    print("\n😊 Analyzing facial expressions...")
    facial_analysis = analyze_facial_expressions(video_path)

    print("\n👁️ Analyzing eye movement...")
    eye_analysis = analyze_eye_movement(video_path)

    analysis_result = {
        "speech_analysis": speech_analysis,
        "facial_expression_analysis": facial_analysis,
        "eye_movement_analysis": eye_analysis,
    }

    conf_result = calculate_confidence_scientific(analysis_result)
    interpretations = interpret_non_verbal_analysis(analysis_result)

    elapsed = time.time() - start_time

    print(f'\n✅ Non-Verbal Analysis Complete in {elapsed:.1f}s')
    print(f'   Confidence: {conf_result["total_confidence_score"]}% ({conf_result["confidence_level"]})')

    # ✅ CLEANUP: Delete temp audio file if we created it
    if temp_audio_created and audio_path and os.path.exists(audio_path):
        try:
            os.remove(audio_path)
            file_size_mb = os.path.getsize(audio_path) / (1024 * 1024) if os.path.exists(audio_path) else 0
            print(f'   🗑️  Temp audio deleted: {os.path.basename(audio_path)} ({file_size_mb:.2f} MB freed)')
        except Exception as e:
            print(f'   ⚠️  Failed to delete temp audio: {str(e)}')

    return {
        'analysis': analysis_result,
        'confidence_score': conf_result['total_confidence_score'],
        'confidence_level': conf_result['confidence_level'],
        'confidence_components': conf_result['confidence_per_metric'],  # ✅ FIXED: Use confidence_per_metric instead of confidence_components
        'confidence_interval': conf_result['confidence_interval'],
        'interpretations': interpretations,
        'processing_time_seconds': round(elapsed, 2)
    }

In [ ]:
def summarize_non_verbal_batch(assessment_results):
    """Ringkasan batch dengan scientific rigor dan transparency"""
    speaking_ratios, pauses, speech_rates = [], [], []
    smiles, eyebrows, eye_contacts, blink_rates = [], [], [], []
    confidence_scores = []
    all_intervals = []

    for item in assessment_results:
        nv = item["result"]["non_verbal_analysis"]

        sp = nv["speech_analysis"]
        speaking_ratios.append(sp["speaking_ratio"])
        pauses.append(sp["number_of_pauses"])
        speech_rates.append(sp["speech_rate_wpm"])

        fc = nv["facial_expression_analysis"]
        smiles.append(fc["average_smile_intensity"])
        eyebrows.append(fc["eyebrow_movement_range"])

        ey = nv["eye_movement_analysis"]
        eye_contacts.append(ey["eye_contact_percentage"])
        blink_rates.append(ey["blink_rate_per_minute"])

        conf_result = calculate_confidence_scientific(nv)
        confidence_scores.append(conf_result["total_confidence_score"])
        all_intervals.append(conf_result["confidence_interval"])

    avg_confidence = round(np.mean(confidence_scores), 2) if confidence_scores else 0
    std_confidence = round(np.std(confidence_scores), 2) if confidence_scores else 0
    max_confidence = round(max(confidence_scores), 2) if confidence_scores else 0
    min_confidence = round(min(confidence_scores), 2) if confidence_scores else 0

    avg_lower = round(np.mean([ci["lower"] for ci in all_intervals]), 2)
    avg_upper = round(np.mean([ci["upper"] for ci in all_intervals]), 2)
    avg_margin = round(np.mean([ci["margin_of_error"] for ci in all_intervals]), 2)

    if avg_confidence >= 80:
        confidence_level = "High"
    elif avg_confidence >= 70:
        confidence_level = "Good"
    elif avg_confidence >= 60:
        confidence_level = "Moderate"
    elif avg_confidence >= 50:
        confidence_level = "Fair"
    else:
        confidence_level = "Low"

    aggregated_data = {
        "speech_analysis": {
            "avg_speaking_ratio": round(np.mean(speaking_ratios), 3),
            "avg_pauses": round(np.mean(pauses), 2),
            "avg_speech_rate": round(np.mean(speech_rates), 2)
        },
        "facial_expression_analysis": {
            "avg_smile_intensity": round(np.mean(smiles), 4),
            "avg_eyebrow_movement_range": round(np.mean(eyebrows), 4)
        },
        "eye_movement_analysis": {
            "avg_eye_contact": round(np.mean(eye_contacts), 2),
            "avg_blink_rate": round(np.mean(blink_rates), 2)
        }
    }

    interpretations = interpret_non_verbal_analysis(aggregated_data)
    summary_text = " ".join([
        interpretations.get("speech_analysis", ""),
        interpretations.get("facial_expression_analysis", ""),
        interpretations.get("eye_movement_analysis", "")
    ])

    poor_performance_count = sum(1 for score in confidence_scores if score < 60)
    poor_performance_percentage = round((poor_performance_count / len(confidence_scores) * 100), 2) if confidence_scores else 0

    performance_level = get_performance_level(avg_confidence)

    confidence_interval = {
        "lower": avg_lower,
        "upper": avg_upper,
        "margin_of_error": avg_margin
    }

    recommendation = get_recommendation(avg_confidence, confidence_interval, interpretations)

    return {
        "overall_performance_status": performance_level,
        "overall_confidence_score": avg_confidence,
        "summary": summary_text,
    }

<b><h2> Fungsi Transkrip Video

In [ ]:
def clean_repetitive_text(text, max_repetitions=3):
    """Remove repetitive patterns at the end of transcription"""
    # Remove excessive repetitions (more than max_repetitions)
    words = text.split()
    if len(words) < 10:
        return text

    # Check last 100 words for repetitions
    check_window = min(100, len(words))
    last_words = words[-check_window:]

    # Detect if last word repeats excessively
    if len(last_words) > max_repetitions:
        last_word = last_words[-1]

        # Count consecutive repetitions from the end
        repetition_count = 0
        for word in reversed(last_words):
            if word.lower() == last_word.lower():
                repetition_count += 1
            else:
                break

        # If repetition exceeds threshold, remove them
        if repetition_count > max_repetitions:
            # Keep only max_repetitions of the repeated word
            words = words[:-repetition_count] + [last_word] * max_repetitions
            print(f'   🧹 Cleaned {repetition_count - max_repetitions} repetitive words')

    # Remove common hallucination patterns
    cleaned_text = ' '.join(words)

    # Pattern: word repeated 5+ times in a row
    cleaned_text = re.sub(r'\b(\w+)(?:\s+\1){4,}\b', r'\1', cleaned_text)

    return cleaned_text.strip()

In [ ]:
def transcribe_video(video_path, language="en"):
    """Transcribe video using faster-whisper with MAXIMUM ACCURACY settings and weighted confidence"""
    try:
        if not os.path.exists(video_path):
            raise Exception(f"Video file not found: {video_path}")

        if not os.access(video_path, os.R_OK):
            raise Exception(f"Video file is not readable: {video_path}")

        file_size = os.path.getsize(video_path) / (1024 * 1024)
        print(f'📁 Video: {os.path.basename(video_path)} ({file_size:.2f} MB)')

        # ✅ LANGUAGE SELECTION
        if language == "id":
            whisper_language = "id"
            initial_prompt = "This is a professional interview in Indonesian (Bahasa Indonesia)."
            print('🌐 Language: Indonesian (Bahasa Indonesia)')
        elif language == "en":
            whisper_language = "en"
            initial_prompt = "This is a professional interview in English."
            print('🌐 Language: English')
        else:
            # Default to English if unknown
            whisper_language = "en"
            initial_prompt = "This is a professional interview in English."
            print(f'⚠️ Unknown language code "{language}", defaulting to English')

        print('🔄 Starting transcription...')
        start_time = time.time()

        # Dynamic parameters based on file size
        if file_size > 30:
            print('   ⚡ Large file - using balanced mode')
            beam_size = 5
            best_of = 5
        else:
            beam_size = 7
            best_of = 7

        # ✅ Optimized VAD parameters
        vad_params = {
            "threshold": 0.3,
            "min_speech_duration_ms": 200,
            "max_speech_duration_s": float('inf'),
            "min_silence_duration_ms": 1500,
            "speech_pad_ms": 500
        }

        # ✅ Transcribe with language parameter
        segments, info = whisper_model.transcribe(
            video_path,
            language=whisper_language,  # ✅ DYNAMIC LANGUAGE
            task="transcribe",
            beam_size=beam_size,
            best_of=best_of,
            patience=2.5,
            length_penalty=0.8,
            repetition_penalty=1.5,
            temperature=0.0,
            compression_ratio_threshold=2.2,
            log_prob_threshold=-0.5,
            no_speech_threshold=0.5,
            condition_on_previous_text=True,
            initial_prompt=initial_prompt,  # ✅ DYNAMIC PROMPT
            word_timestamps=True,
            vad_filter=True,
            vad_parameters=vad_params
        )

        # Collect segments with confidence scores
        print('   📝 Collecting segments...')
        transcription_text = ""
        segments_list = list(segments)

        confidence_scores = []
        segment_details = []

        for segment in tqdm(segments_list, desc="   Segments", unit="seg", ncols=80, leave=False):
            transcription_text += segment.text + " "

            # Calculate confidence from log probability
            confidence = segment.avg_logprob
            # ✅ FORMULA: konversi log prob (-inf to 0) ke percentage (0-100)
            confidence_percent = round(min(100, max(0, np.exp(confidence) * 100)), 2)
            confidence_scores.append(confidence_percent)

            segment_details.append({
                "text": segment.text.strip(),
                "start": round(segment.start, 2),
                "end": round(segment.end, 2),
                "duration": round(segment.end - segment.start, 2),
                "confidence": confidence_percent
            })

        transcription_text = transcription_text.strip()

        if not transcription_text:
            print('   ⚠️  No speech detected')
            return "[No speech detected in video]", 0.0

        # ============================================================
        # ✅ WEIGHTED CONFIDENCE CALCULATION
        # ============================================================
        if confidence_scores:
            # 1. Calculate weighted confidence by segment duration
            segment_durations = [seg.end - seg.start for seg in segments_list]
            total_duration = sum(segment_durations)

            weighted_confidence = sum(
                conf * (duration / total_duration)
                for conf, duration in zip(confidence_scores, segment_durations)
            )

            # 2. Quality-based boost
            word_count = len(transcription_text.split())
            min_conf = min(confidence_scores)
            max_conf = max(confidence_scores)
            conf_variance = max_conf - min_conf

            boost = 1.0  # Default: no boost
            boost_reason = ""

            # Boost conditions (only for reliable transcriptions)
            if word_count >= 100 and min_conf >= 60:  # Excellent
                boost = 1.35
                boost_reason = "Excellent: Long & very high quality"
            elif word_count >= 100 and min_conf >= 50:  # Very Good
                boost = 1.30
                boost_reason = "Very Good: Long & consistent"
            elif word_count >= 50 and min_conf >= 60:  # Good
                boost = 1.25
                boost_reason = "Good: Medium length, high quality"
            elif word_count >= 50 and conf_variance <= 25:  # Stable
                boost = 1.20
                boost_reason = "Stable: Consistent confidence"
            elif word_count >= 30 and min_conf >= 65:  # High Quality Short
                boost = 1.15
                boost_reason = "Short but very high quality"

            # Apply boost (cap at 98%)
            avg_confidence = round(min(98, weighted_confidence * boost), 2)

            # Show boost info if applied
            if boost > 1.0:
                print(f'   🔼 Quality boost: +{((boost-1)*100):.1f}% ({boost_reason})')
        else:
            avg_confidence = 0.0

        # Clean repetitive text
        original_length = len(transcription_text)
        transcription_text = clean_repetitive_text(transcription_text, max_repetitions=3)

        if len(transcription_text) < original_length:
            print(f'   🧹 Cleaned: {original_length} → {len(transcription_text)} chars')

        total_time = time.time() - start_time
        words = transcription_text.split()

        # Display results
        print(f'   ✅ Completed in {total_time:.1f}s | {len(segments_list)} segments | {len(words)} words')
        print(f'   🎯 Transcription Confidence: {avg_confidence}% {"✅" if avg_confidence >= 70 else "⚠️" if avg_confidence >= 50 else "❌"}')

        if confidence_scores:
            min_conf = min(confidence_scores)
            max_conf = max(confidence_scores)
            simple_avg = round(sum(confidence_scores) / len(confidence_scores), 2)
            print(f'   📊 Confidence Range: {min_conf}% - {max_conf}%')
            print(f'   📊 Simple Average: {simple_avg}% | Weighted: {avg_confidence}%')

        # Cleanup
        gc.collect()

        return transcription_text, avg_confidence, min_conf, max_conf, simple_avg

    except Exception as e:
        print(f'   ❌ Error: {str(e)}')
        gc.collect()
        raise Exception(f"Transcription failed: {str(e)}")

<b><h2> Fungsi Translate to Indonesia

In [ ]:
def translate_to_indonesian(text):
    """Translate English text to Indonesian using DeepL"""
    if not translator:
        print('   ⚠️  Translation skipped (no API key)')
        return {
            "translated_text": "[Translation not available]"
        }

    try:
        max_chunk_size = 5000

        if len(text) <= max_chunk_size:
            result = translator.translate_text(text, source_lang="EN", target_lang="ID")
            translated_text = result.text
        else:
            sentences = text.split('. ')
            chunks = []
            current_chunk = ""

            for sentence in sentences:
                if len(current_chunk) + len(sentence) + 2 <= max_chunk_size:
                    current_chunk += sentence + ". "
                else:
                    if current_chunk:
                        chunks.append(current_chunk)
                    current_chunk = sentence + ". "

            if current_chunk:
                chunks.append(current_chunk)

            translated_chunks = []
            for chunk in chunks:
                result = translator.translate_text(chunk, source_lang="EN", target_lang="ID")
                translated_chunks.append(result.text)

            translated_text = " ".join(translated_chunks)

        print(f'   ✅ Translation: {len(text)} → {len(translated_text)} chars')

        return {
            "translated_text": translated_text
        }

    except Exception as e:
        print(f'   ❌ Translation failed: {str(e)}')
        return {
            "translated_text": f"[Translation failed: {str(e)}]"
        }

In [ ]:
def translate_to_english(text):
    """Translate Indonesian text to English using DeepL"""
    if not translator:
        print('   ⚠️  Translation skipped (no API key)')
        return {
            "translated_text": "[Translation not available]"
        }

    try:
        max_chunk_size = 5000

        if len(text) <= max_chunk_size:
            result = translator.translate_text(text, source_lang="ID", target_lang="EN-US")
            translated_text = result.text
        else:
            # Split by sentences for Indonesian
            sentences = text.split('. ')
            chunks = []
            current_chunk = ""

            for sentence in sentences:
                if len(current_chunk) + len(sentence) + 2 <= max_chunk_size:
                    current_chunk += sentence + ". "
                else:
                    if current_chunk:
                        chunks.append(current_chunk)
                    current_chunk = sentence + ". "

            if current_chunk:
                chunks.append(current_chunk)

            translated_chunks = []
            for chunk in chunks:
                result = translator.translate_text(chunk, source_lang="ID", target_lang="EN-US")
                translated_chunks.append(result.text)

            translated_text = " ".join(translated_chunks)

        print(f'   ✅ Translation: {len(text)} → {len(translated_text)} chars')

        return {
            "translated_text": translated_text
        }

    except Exception as e:
        print(f'   ❌ Translation failed: {str(e)}')
        return {
            "translated_text": f"[Translation failed: {str(e)}]"
        }

<b><h2> Initialize HuggingFace

In [ ]:
# ✅ HuggingFace API Token
# HF_TOKEN = "token token"
# os.environ["HF_TOKEN"] = HF_TOKEN

# Initialize Inference Client
print('📥 Initializing HuggingFace Inference API...')
print('ℹ️  Using meta-llama/Llama-3.1-8B-Instruct via Inference API')

client = InferenceClient(api_key=HF_TOKEN)

print('✅ Inference API initialized successfully\n')

def evaluate_with_llm(transcription_text: str, question: str, position_id: int):
    """
    Evaluate interview answer using deterministic LLM evaluation with confidence scoring.
    NOW WITH LOG PROBABILITIES SUPPORT + BOOST SYSTEM for confidence enhancement.
    """
    try:
        # Construct evaluation prompt
        user_message = f"""You are an expert interview evaluator about programming and machine learning. You must provide objective, consistent scores based on explicit criteria and formulas.

**INTERVIEW QUESTION**: "{question}"

**CANDIDATE'S ANSWER**: "{transcription_text}"

**EVALUATION RUBRIC WITH FORMULAS**:

1. **KUALITAS JAWABAN (Quality of Answer)** [1-100]:

   Base Score Formula:
   - If answer addresses question with examples/details: BASE = 85
   - If answer addresses question adequately: BASE = 75
   - If answer is brief but relevant: BASE = 65
   - If answer is unclear/irrelevant: BASE = 45

   Adjustments:
   - Provides specific examples: +5 to +15
   - Shows deep understanding: +5 to +10
   - Lacks depth: -10 to -20
   - Vague/incomplete: -15 to -25

   MINIMUM for acceptable answers: 70

2. **KOHERENSI (Coherence)** [1-100]:

   Formula:
   - Logical flow, well-structured: BASE = 85
   - Adequate structure: BASE = 75
   - Some inconsistency: BASE = 65
   - Disorganized: BASE = 45

   Adjustments:
   - Clear progression: +5 to +10
   - Smooth transitions: +5 to +10
   - Contradictory statements: -15 to -25
   - Jumps between topics: -10 to -20

   MINIMUM for coherent answers: 70

3. **RELEVANSI (Relevance)** [1-100]:

   Formula:
   - Directly answers the question: BASE = 85
   - Addresses most aspects: BASE = 75
   - Partially relevant: BASE = 65
   - Off-topic: BASE = 45

   Adjustments:
   - Covers all question aspects: +10 to +20
   - Provides context: +5 to +10
   - Deviates from topic: -15 to -25

   MINIMUM for on-topic answers: 70

**CALCULATION STEPS**:
1. Analyze the answer content and structure
2. Calculate base scores using formulas
3. Apply adjustments

**OUTPUT FORMAT** (JSON only, no explanation):
{{
  "kualitas_jawaban": <integer 1-100>,
  "koherensi": <integer 1-100>,
  "relevansi": <integer 1-100>,
  "analysis": "<2-3 sentence justification with reasoning>"
}}
"""

        # Calculate word count for boost system
        word_count = len(transcription_text.split())

        print(f'│ 🤖 LLM Evaluation...')
        print(f'│ 📝 Answer length: {len(transcription_text)} chars ({word_count} words)')

        # ✅ API Call with logprobs enabled
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a highly objective interview evaluator about programming and machine learning. Always respond with valid JSON only, no markdown."
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            max_tokens=600,
            temperature=0.1,
            top_p=0.9,
            logprobs=True,        # ✅ ENABLE LOG PROBABILITIES
            top_logprobs=3        # ✅ Get top 3 alternative tokens for each position
        )

        # Extract response text
        response = completion.choices[0].message.content.strip()
        response = re.sub(r'^```json\s*', '', response)
        response = re.sub(r'\s*```$', '', response)

        print(f'│ 📨 API Response received ({len(response)} chars)')

        # ============================================================
        # ✅ EXTRACT LOGPROBS DATA (NEW!)
        # ============================================================
        logprobs_data = None
        raw_token_confidence = None
        raw_avg_probability = None

        try:
            if hasattr(completion.choices[0], 'logprobs') and completion.choices[0].logprobs:
                logprobs_obj = completion.choices[0].logprobs

                # Check if content exists
                if hasattr(logprobs_obj, 'content') and logprobs_obj.content:
                    logprobs_data = logprobs_obj.content

                    # Calculate average log probability
                    token_logprobs = [token.logprob for token in logprobs_data if hasattr(token, 'logprob')]

                    if token_logprobs:
                        avg_logprob = sum(token_logprobs) / len(token_logprobs)
                        # Convert log probability to percentage confidence
                        raw_avg_probability = math.exp(avg_logprob)
                        raw_token_confidence = round(raw_avg_probability * 100, 2)

                        print(f'│ 🎯 Raw Logprobs extracted: {len(token_logprobs)} tokens')
                        print(f'│ 📊 Avg log prob: {avg_logprob:.4f}')
                        print(f'│ ✨ Raw token confidence: {raw_token_confidence}%')
                    else:
                        print(f'│ ⚠️  Logprobs available but no token data')
                else:
                    print(f'│ ⚠️  Logprobs object has no content')
            else:
                print(f'│ ⚠️  No logprobs in API response (may not be supported)')
        except Exception as logprob_error:
            print(f'│ ⚠️  Logprobs extraction failed: {str(logprob_error)}')
            # Continue without logprobs - non-critical feature

        # Extract JSON from response
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            evaluation = json_module.loads(json_str)
        else:
            raise ValueError("No valid JSON found in API response")

        # Validate scores
        required_keys = ['kualitas_jawaban', 'koherensi', 'relevansi']
        for key in required_keys:
            if key not in evaluation:
                raise ValueError(f"Missing required key: {key}")
            evaluation[key] = max(1, min(100, int(evaluation[key])))

        # ============================================================
        # DISPLAY RESULTS
        # ============================================================
        print(f'│ 📊 LLM Scores:')
        print(f'│    • Quality: {evaluation["kualitas_jawaban"]}/100')
        print(f'│    • Coherence: {evaluation["koherensi"]}/100')
        print(f'│    • Relevance: {evaluation["relevansi"]}/100')

        # Calculate total score (using boosted model confidence)
        total = round((
            evaluation["kualitas_jawaban"] +
            evaluation["koherensi"] +
            evaluation["relevansi"]
        ) / 3)

        print(f'│ ✅ Total Score: {total}/100')

        # ✅ Return with logprobs data and boost info
        result = {
            "scores": {
                "kualitas_jawaban": evaluation["kualitas_jawaban"],
                "koherensi": evaluation["koherensi"],
                "relevansi": evaluation["relevansi"],
                "confidence_score": raw_token_confidence
            },
            "total": total,
            "analysis": evaluation.get('analysis', 'No analysis provided'),
            # 🆕 Logprobs data
            "logprobs_confidence": raw_token_confidence,
            "logprobs_probability": raw_avg_probability,
            "logprobs_available": logprobs_data is not None,
        }

        return result

    except Exception as e:
        print(f'│ ⚠️  LLM evaluation failed: {str(e)}')
        print(f'│ 🔄 Falling back to rule-based assessment...')

        # Fallback assessment
        word_count = len(transcription_text.split())

        # Simple heuristic scoring
        if word_count > 100:
            quality_score = 75
            coherence_score = 70
            relevance_score = 70
            model_confidence = 60
        elif word_count > 50:
            quality_score = 65
            coherence_score = 65
            relevance_score = 65
            model_confidence = 55
        elif word_count > 20:
            quality_score = 55
            coherence_score = 55
            relevance_score = 55
            model_confidence = 50
        else:
            quality_score = 40
            coherence_score = 35
            relevance_score = 35
            model_confidence = 50

        total = round((quality_score + coherence_score + relevance_score) / 3)

        return {
            "scores": {
                "kualitas_jawaban": quality_score,
                "koherensi": coherence_score,
                "relevansi": relevance_score,
                "confidence_score": model_confidence
            },
            "total": total,
            "analysis": f"Fallback rule-based assessment (word count: {word_count}). LLM evaluation unavailable: {str(e)}",
            # Fallback has no logprobs or boost
            "logprobs_confidence": None,
            "logprobs_probability": None,
            "logprobs_available": False,
        }

In [ ]:
def summarize_llm_analysis_batch(assessment_results):
    """Generate overall summary from all assessments using LLM with logprobs support"""
    try:
        if not assessment_results:
            return {
                "kesimpulan_llm": "Tidak ada hasil penilaian yang tersedia.",
                "rata_rata_confidence_score": 0,
                "avg_total_llm": 0,
                "avg_logprobs_confidence": 0,
                "final_score_llm": 0
            }

        # Calculate averages
        confidence_scores = []
        total_scores = []
        logprobs_confidences = []  # 🆕 NEW

        for result in assessment_results:
            assessment = result.get('result', {}).get('penilaian', {})
            confidence_scores.append(assessment.get('confidence_score', 0))
            total_scores.append(assessment.get('total', 0))
            
            # 🆕 Extract logprobs confidence if available
            if 'logprobs_confidence' in result.get('result', {}).get('penilaian', {}):
                lp_conf = result['result']['penilaian'].get('logprobs_confidence')
                if lp_conf is not None:
                    logprobs_confidences.append(lp_conf)

        avg_confidence = round(sum(confidence_scores) / len(confidence_scores)) if confidence_scores else 0
        avg_total = round(sum(total_scores) / len(total_scores)) if total_scores else 0
        
        # 🆕 Calculate average logprobs confidence
        avg_logprobs_confidence = None
        if logprobs_confidences:
            avg_logprobs_confidence = round(sum(logprobs_confidences) / len(logprobs_confidences), 2)

        # Determine final score
        projectScore = 100
        final_score = projectScore * 0.7 + avg_total * 0.3

        # Prepare assessment summary
        summary_lines = []
        for idx, result in enumerate(assessment_results, 1):
            assessment = result.get('result', {}).get('penilaian', {})
            summary_lines.append(
                f"Video {idx}: Total {assessment.get('total', 0)}/100 "
                f"(Quality: {assessment.get('kualitas_jawaban', 0)}, "
                f"Coherence: {assessment.get('koherensi', 0)}, "
                f"Relevance: {assessment.get('relevansi', 0)})"
            )

        assessment_summary = "\n".join(summary_lines)

        # Generate LLM summary prompt
        user_message = f"""Based on the following interview assessment results, provide a comprehensive summary in Indonesian (2-3 paragraphs, ~150-200 words).

**ASSESSMENT RESULTS**:
{assessment_summary}

**AVERAGES**:
- Average Total Score: {avg_total}/100

**INSTRUCTIONS**:
1. Summarize the candidate's overall performance across all videos
2. Highlight strengths and areas for improvement
3. Be objective and constructive

Respond with plain text summary only (no JSON, no markdown formatting)."""

        print(f'\n{"="*70}')
        print(f'🤖 Generating LLM Summary...')
        print(f'📊 Processing {len(assessment_results)} video assessments')
        print(f'📈 Average Score: {avg_total}/100')
        if avg_logprobs_confidence is not None:
            print(f'✨ Avg Logprobs Confidence: {avg_logprobs_confidence}%')
        print(f'{"="*70}')
        print(f'🤖 Calling LLM to generate summary...')

        # ✅ API Call with logprobs
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert interview analyst. Respond with plain text only, no JSON."
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            max_tokens=500,
            temperature=0.3,
            top_p=0.9,
            logprobs=True,      # 🆕 ENABLE logprobs
            top_logprobs=3      # 🆕 Get alternatives
        )

        # Extract summary
        kesimpulan_llm = completion.choices[0].message.content.strip()
        kesimpulan_llm = re.sub(r'^```.*?\n', '', kesimpulan_llm)
        kesimpulan_llm = re.sub(r'\n```$', '', kesimpulan_llm)

        # 🆕 Extract summary logprobs
        summary_logprobs_confidence = None
        try:
            if hasattr(completion.choices[0], 'logprobs') and completion.choices[0].logprobs:
                logprobs_obj = completion.choices[0].logprobs
                if hasattr(logprobs_obj, 'content') and logprobs_obj.content:
                    token_logprobs = [token.logprob for token in logprobs_obj.content if hasattr(token, 'logprob')]
                    if token_logprobs:
                        avg_logprob = sum(token_logprobs) / len(token_logprobs)
                        summary_logprobs_confidence = round(math.exp(avg_logprob) * 100, 2)
                        print(f'✨ Summary logprobs confidence: {summary_logprobs_confidence}%')
        except Exception as e:
            print(f'⚠️  Summary logprobs extraction failed: {str(e)}')

        print(f'✅ LLM Summary generated successfully')
        print(f'   Length: {len(kesimpulan_llm)} characters')
        print(f'{" ="*70}\n')

        return {
            "kesimpulan_llm": kesimpulan_llm,
            "rata_rata_confidence_score": avg_confidence,
            "avg_total_llm": avg_total,
            "final_score_llm": final_score,
            # 🆕 NEW: Logprobs data
            "avg_logprobs_confidence": avg_logprobs_confidence,  # Average across all videos
        }

    except Exception as e:
        print(f'❌ LLM summary generation failed: {str(e)}')
        print(f'🔄 Using fallback summary...')
        
        return {
            "kesimpulan_llm": f"Kandidat menunjukkan performa dengan rata-rata skor {avg_total}/100. "
                             f"(LLM summary unavailable: {str(e)})",
            "rata_rata_confidence_score": avg_confidence,
            "avg_total_llm": avg_total,
            "avg_logprobs_confidence": None,
        }

<b><h2> Google Drive Download

In [ ]:
# ===== HELPER: Download video from Google Drive =====
import gdown
import requests
from urllib.parse import urlparse, parse_qs

def download_video_from_google_drive(video_url, dest_folder):
    """Download video from Google Drive URL"""
    try:
        # Extract file ID from Google Drive URL
        if 'drive.google.com' in video_url:
            # Format 1: https://drive.google.com/file/d/FILE_ID/view?usp=...
            if '/file/d/' in video_url:
                file_id = video_url.split('/file/d/')[1].split('/')[0]
            # Format 2: https://drive.google.com/open?id=FILE_ID
            elif 'id=' in video_url:
                parsed = urlparse(video_url)
                file_id = parse_qs(parsed.query)['id'][0]
            else:
                raise ValueError(f"Unsupported Google Drive URL format")
            
            # Generate download URL
            download_url = f"https://drive.google.com/uc?id={file_id}&export=download"
            
            # Generate safe filename
            safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.mp4"
            dest_path = os.path.join(dest_folder, safe_name)
            
            print(f"      📥 Downloading from Google Drive (ID: {file_id[:20]}...)")
            
            # Download with gdown
            gdown.download(download_url, dest_path, quiet=False)
            
            # Verify file exists and has content
            if not os.path.exists(dest_path) or os.path.getsize(dest_path) == 0:
                raise ValueError("Downloaded file is empty or doesn't exist")
            
            file_size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"      ✅ Downloaded: {safe_name} ({file_size_mb:.2f} MB)")
            
            return safe_name, dest_path
        
        else:
            # Direct URL download (fallback)
            safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.mp4"
            dest_path = os.path.join(dest_folder, safe_name)
            
            print(f"      📥 Downloading from direct URL")
            response = requests.get(video_url, stream=True, timeout=300)
            response.raise_for_status()
            
            with open(dest_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            file_size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"      ✅ Downloaded: {safe_name} ({file_size_mb:.2f} MB)")
            
            return safe_name, dest_path
    
    except Exception as e:
        print(f"      ❌ Download failed: {str(e)}")
        raise

In [ ]:
# ===== BACKGROUND THREAD: Download and Process Videos =====
def download_and_process_videos(session_id, candidate_name, interviews, language, base_url):
    """
    Background thread: Download videos from Google Drive URLs, then process identically to /upload endpoint.
    """
    print(f"\n🔽 [Thread-{session_id[:8]}] Starting video downloads...")
    
    try:
        uploaded_videos = []
        
        # PHASE 1: Download all videos
        print(f'\n📥 Downloading {len(interviews)} video(s) from Google Drive...')
        for idx, interview in enumerate(interviews, 1):
            try:
                # Update download progress
                with processing_lock:
                    processing_status[session_id]['message'] = f'Downloading video {idx}/{len(interviews)}...'
                    processing_status[session_id]['progress'] = f'{idx}/{len(interviews)}'
                
                # Extract fields from interview
                position_id = interview.get('positionId', idx)
                question = interview.get('question', '')
                is_video_exist = interview.get('isVideoExist', False)
                video_url = interview.get('recordedVideoUrl', '')
                
                print(f'\n   📹 Video {idx}/{len(interviews)}:')
                print(f'      Position ID: {position_id}')
                print(f'      Question: {question[:60]}{"..." if len(question) > 60 else ""}')
                print(f'      Video exists: {is_video_exist}')
                print(f'      URL: {video_url[:80]}{"..." if len(video_url) > 80 else ""}')
                
                # Validate
                if not question:
                    print(f'      ⚠️ Missing question, skipping')
                    uploaded_videos.append({
                        'positionId': position_id,
                        'question': '',
                        'isVideoExist': False,
                        'recordedVideoUrl': None,
                        'error': 'Missing question field'
                    })
                    continue
                
                if not is_video_exist or not video_url:
                    print(f'      ⚠️ No video URL, skipping')
                    uploaded_videos.append({
                        'positionId': position_id,
                        'question': question,
                        'isVideoExist': False,
                        'recordedVideoUrl': None,
                        'error': 'No video URL provided'
                    })
                    continue
                
                # Download video from Google Drive
                safe_name, dest_path = download_video_from_google_drive(video_url, UPLOAD_DIR)
                
                # Create local file URL (same as /upload endpoint)
                file_url = f"{base_url}/uploads/{safe_name}"
                
                uploaded_videos.append({
                    'positionId': position_id,
                    'question': question,
                    'isVideoExist': True,
                    'recordedVideoUrl': file_url,
                    'filename': safe_name
                })
                
            except Exception as e:
                print(f'      ❌ Failed to download video {idx}: {str(e)}')
                uploaded_videos.append({
                    'positionId': interview.get('positionId', idx),
                    'question': interview.get('question', ''),
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': str(e)
                })
        
        successful_downloads = len([v for v in uploaded_videos if v['isVideoExist']])
        print(f"\n✅ Download complete: {successful_downloads}/{len(interviews)} successful")
        
        # PHASE 2: Update status to processing (same as /upload endpoint)
        with processing_lock:
            processing_status[session_id] = {
                'status': 'processing',
                'progress': '0/' + str(len(uploaded_videos)),
                'message': 'Starting transcription...',
                'uploaded_videos': len(uploaded_videos)
            }
        
        # PHASE 3: Process transcriptions (IDENTICAL to /upload endpoint)
        print(f'\n🔄 Starting transcription process (identical to /upload endpoint)...')
        process_transcriptions_sync(session_id, candidate_name, uploaded_videos, base_url, language)
    
    except Exception as e:
        error_detail = traceback.format_exc()
        print(f"❌ Download thread error:\n{error_detail}")
        
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': error_detail
            }

<b><h2> Final Processing

In [ ]:
def process_transcriptions_sync(session_id: str, candidate_name: str, uploaded_videos: list, base_url: str, language: str = "en"):
    """Background transcription processing WITH COMPREHENSIVE LOGGING"""

    # ⭐ SETUP LOGGING TO FILE
    log_file = f'session_{session_id}.log'
    log_handle = open(log_file, 'w', encoding='utf-8', buffering=1)

    def log_print(msg):
        """Print to both console and log file"""
        print(msg, flush=True)
        log_handle.write(msg + '\n')
        log_handle.flush()

    try:
        log_print(f'\n{"="*70}')
        log_print(f'🎙️  SESSION: {session_id}')
        log_print(f'👤 CANDIDATE: {candidate_name}')
        log_print(f'🌐 LANGUAGE: {"English" if language == "en" else "Indonesian" if language == "id" else language}')
        log_print(f'📹 VIDEOS: {len(uploaded_videos)}')
        log_print(f'📝 LOG FILE: {log_file}')
        log_print(f'{"="*70}\n')

        transcriptions = []
        assessment_results = []

        with processing_lock:
            processing_status[session_id] = {'status': 'processing', 'progress': '0/0'}

        # Process each video
        for idx, interview in enumerate(uploaded_videos, 1):
            log_print(f'\n{"─"*70}')
            log_print(f'Processing video {idx}/{len(uploaded_videos)}')
            log_print(f'{"─"*70}')

            if not interview.get('isVideoExist') or not interview.get('recordedVideoUrl'):
                log_print(f'⚠️ Video {idx} - No video exists or no URL')
                transcriptions.append({
                    'positionId': interview['positionId'],
                    'error': interview.get('error', 'Video upload failed')
                })
                continue

            position_id = interview['positionId']
            video_url = interview['recordedVideoUrl']
            question = interview.get('question', '')

            try:
                log_print(f'\n┌─ Video {position_id}/{len(uploaded_videos)} ─{"─"*50}┐')
                if question:
                    log_print(f'│ ❓ Question: {question[:60]}{"..." if len(question) > 60 else ""}')

                local_file = get_local_file_path(video_url)
                if not local_file:
                    raise Exception(f"Local file not found")

                log_print(f'│ 📁 Local file: {local_file}')
                log_print(f'│ 📏 File exists: {os.path.exists(local_file)}')

                file_size_mb = os.path.getsize(local_file) / (1024 * 1024)
                log_print(f'│ 📊 File size: {file_size_mb:.1f} MB')

                with processing_lock:
                    processing_status[session_id] = {
                        'status': 'processing',
                        'progress': f'{position_id}/{len(uploaded_videos)}',
                        'current_video': position_id,
                        'message': f'Processing video {position_id}/{len(uploaded_videos)}...'
                    }

                video_start = time.time()

                # Step 1: Transcribe
                log_print(f'│ 1️⃣  TRANSCRIPTION ({file_size_mb:.1f} MB)')
                try:
                    transcription_text, avg_confidence, min_conf, max_conf, simple_avg = transcribe_video(local_file, language=language)
                    transcribe_time = time.time() - video_start
                    log_print(f'│    ✅ Transcription completed')
                    log_print(f'│    🎯 Transcription Confidence: {avg_confidence}%')
                    log_print(f'│    📝 Text length: {len(transcription_text)} chars')
                except Exception as e:
                    log_print(f'│    ❌ Transcription ERROR: {str(e)}')
                    raise

                # Step 2: Translate (conditional based on language)
                log_print(f'│ 2️⃣  TRANSLATION')
                try:
                    translate_start = time.time()

                    if language == "en":
                        # English → Indonesian
                        translation_result = translate_to_indonesian(transcription_text)
                        transcription_en = transcription_text  # Original is English
                        transcription_id = translation_result['translated_text']  # Translated to Indonesian
                        log_print(f'│    🌐 Direction: English → Indonesian')
                    elif language == "id":
                        # Indonesian → English
                        translation_result = translate_to_english(transcription_text)
                        transcription_id = transcription_text  # Original is Indonesian
                        transcription_en = translation_result['translated_text']  # Translated to English
                        log_print(f'│    🌐 Direction: Indonesian → English')
                    else:
                        # Default: assume English
                        translation_result = translate_to_indonesian(transcription_text)
                        transcription_en = transcription_text
                        transcription_id = translation_result['translated_text']
                        log_print(f'│    ⚠️  Unknown language, defaulting to English → Indonesian')

                    translate_time = time.time() - translate_start
                    log_print(f'│    ✅ Translation completed in {translate_time:.1f}s')
                    log_print(f'│    📝 EN length: {len(transcription_en)} chars')
                    log_print(f'│    📝 ID length: {len(transcription_id)} chars')
                except Exception as e:
                    log_print(f'│    ❌ Translation ERROR: {str(e)}')
                    raise

                # Step 3: Cheating Detection
                log_print(f'│ 2️⃣½ CHEATING DETECTION')
                try:
                    cheating_start = time.time()
                    cheating_result = advanced_cheating_detection(local_file, transcription_en)
                    cheating_time = time.time() - cheating_start
                    log_print(f'│    ✅ Cheating detection completed in {cheating_time:.1f}s')
                except Exception as e:
                    log_print(f'│    ❌ Cheating detection ERROR: {str(e)}')
                    raise

                # Step 4: Non-Verbal Analysis
                log_print(f'│ 2️⃣¾ NON-VERBAL ANALYSIS')
                try:
                    non_verbal_start = time.time()
                    non_verbal_result = analyze_interview_video_with_confidence(
                        video_path=local_file,
                        audio_path=None
                    )
                    non_verbal_time = time.time() - non_verbal_start
                    log_print(f'│    ✅ Non-verbal analysis completed in {non_verbal_time:.1f}s')
                    log_print(f'│    📊 Non-Verbal Confidence: {non_verbal_result["confidence_score"]}%')
                except Exception as e:
                    log_print(f'│    ❌ Non-verbal analysis ERROR: {str(e)}')
                    raise

                # Step 5: LLM Evaluation (always use English text for better accuracy)
                log_print(f'│ 3️⃣  AI ASSESSMENT')
                try:
                    llm_start = time.time()
                    llm_evaluation = evaluate_with_llm(transcription_en, question, position_id)  # ✅ Use English version
                    llm_time = time.time() - llm_start
                    log_print(f'│    ✅ LLM evaluation completed in {llm_time:.1f}s')
                    log_print(f'│    📊 Total Score: {llm_evaluation["total"]}/100')
                except Exception as e:
                    log_print(f'│    ❌ LLM evaluation ERROR: {str(e)}')
                    raise

                # Step 6: Save transcription file
                log_print(f'│ 4️⃣  SAVING FILES')
                trans_fname = f"transcription_pos{position_id}_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.txt"
                trans_path = os.path.join(TRANSCRIPTION_DIR, trans_fname)

                with open(trans_path, 'w', encoding='utf-8') as f:
                    f.write(f"Candidate: {candidate_name}\n")
                    f.write(f"Position ID: {position_id}\n")
                    f.write(f"Question: {question}\n")
                    f.write(f"Video URL: {video_url}\n")
                    f.write(f"Language: {language}\n")  # ✅ Added language info
                    f.write(f"Transcribed at: {datetime.now(timezone.utc).isoformat()}\n")
                    f.write(f"\n{'='*50}\n")

                    if language == "id":
                        f.write(f"INDONESIAN TRANSCRIPTION (Original):\n")
                        f.write(f"{'='*50}\n\n")
                        f.write(transcription_id)
                        f.write(f"\n\n{'='*50}\n")
                        f.write(f"ENGLISH TRANSLATION:\n")
                        f.write(f"{'='*50}\n\n")
                        f.write(transcription_en)
                    else:
                        f.write(f"ENGLISH TRANSCRIPTION (Original):\n")
                        f.write(f"{'='*50}\n\n")
                        f.write(transcription_en)
                        f.write(f"\n\n{'='*50}\n")
                        f.write(f"INDONESIAN TRANSLATION:\n")
                        f.write(f"{'='*50}\n\n")
                        f.write(transcription_id)

                    f.write(f"\n\nTranscription Confidence: {avg_confidence}%\n")

                log_print(f'│    ✅ Transcription file saved: {trans_fname}')

                transcription_url = f"{base_url}/transcriptions/{trans_fname}"

                # Build assessment
                words = transcription_en.split()

                assessment = {
                    "penilaian": {
                        "confidence_score": llm_evaluation['scores']['confidence_score'],
                        "kualitas_jawaban": llm_evaluation['scores']['kualitas_jawaban'],
                        "relevansi": llm_evaluation['scores']['relevansi'],
                        "koherensi": llm_evaluation['scores']['koherensi'],
                        "analisis_llm": llm_evaluation['analysis'],
                        "total": llm_evaluation['total'],
                        # 🆕 NEW: Add logprobs data
                        "logprobs_confidence": llm_evaluation.get('logprobs_confidence'),
                        "logprobs_probability": llm_evaluation.get('logprobs_probability'),
                        "logprobs_available": llm_evaluation.get('logprobs_available', False)
                    },
                    "cheating_detection": cheating_result.get('cheating_status', 'Tidak'),
                    "cheating_score": cheating_result.get('cheating_score', 0),
                    "cheating_confidence_score": cheating_result.get('confidence_score', 0),
                    "non_verbal_analysis": non_verbal_result['analysis'],
                    "non_verbal_confidence_score": non_verbal_result['confidence_score'],
                    "transkripsi_en": transcription_en,
                    "transkripsi_id": transcription_id,
                    "transkripsi_confidence": avg_confidence,
                    "transkripsi_min_confidence": min_conf,
                    "transkripsi_max_confidence": max_conf,
                    "transkripsi_simple_average": simple_avg,
                    "metadata": {
                        "word_count": len(words),
                        "processed_at": datetime.now(timezone.utc).isoformat(),
                        # 🆕 NEW: Logprobs metadata
                        "logprobs_enabled": True
                    }
                }

                assessment_results.append({
                    "id": position_id,
                    "question": question,
                    "result": assessment
                })
                log_print(f'│    ✅ Assessment added to results (total: {len(assessment_results)})')

                transcriptions.append({
                    'positionId': position_id,
                    'question': question,
                    'videoUrl': video_url,
                    'transcription': transcription_en,
                    'transcription_id': transcription_id,
                    'transcriptionUrl': transcription_url,
                    'transcriptionFile': trans_fname,
                    'assessment': assessment
                })

                # Delete video
                if os.path.exists(local_file):
                    os.remove(local_file)
                    log_print(f'│ 🗑️  Video deleted ({file_size_mb:.1f} MB freed)')

                total_time = time.time() - video_start
                log_print(f'│ ⏱️  Total: {total_time:.1f}s')
                log_print(f'└─{"─"*68}┘')

                gc.collect()

            except Exception as e:
                log_print(f'│ ❌ ERROR processing video {position_id}: {str(e)}')
                log_print(f'│ 📋 Traceback:')
                for line in traceback.format_exc().split('\n'):
                    log_print(f'│    {line}')
                log_print(f'└─{"─"*68}┘')

                transcriptions.append({
                    'positionId': position_id,
                    'question': question,
                    'videoUrl': video_url,
                    'error': str(e)
                })

        # ============================================================================
        # AGGREGATE ANALYSIS
        # ============================================================================
        log_print(f'\n{"="*70}')
        log_print(f'📊 STARTING AGGREGATE ANALYSIS')
        log_print(f'{"="*70}')
        log_print(f'Assessment Results Count: {len(assessment_results)}')

        if len(assessment_results) == 0:
            log_print(f'⚠️ WARNING: No assessment results! Cannot create aggregate analysis.')
            log_print(f'   Total transcriptions: {len(transcriptions)}')
            log_print(f'   Transcriptions with errors: {sum(1 for t in transcriptions if "error" in t)}')

        # 1. Aggregate Cheating
        try:
            log_print(f'\n🚨 Calculating aggregate cheating analysis...')
            aggregate_cheating = calculate_aggregate_cheating_analysis(assessment_results)
            log_print(f'✅ Aggregate cheating completed')
        except Exception as e:
            log_print(f'❌ ERROR in aggregate_cheating: {str(e)}')
            log_print(f'   Traceback: {traceback.format_exc()}')
            aggregate_cheating = {
                "overall_cheating_status": "Error",
                "error": str(e)
            }

        # 2. Aggregate Non-Verbal
        try:
            log_print(f'\n👀 Calculating aggregate non-verbal...')
            aggregate_non_verbal = summarize_non_verbal_batch(assessment_results)
            log_print(f'✅ Aggregate non-verbal completed')
        except Exception as e:
            log_print(f'❌ ERROR in aggregate_non_verbal: {str(e)}')
            log_print(f'   Traceback: {traceback.format_exc()}')
            aggregate_non_verbal = {"error": str(e)}

        # 3. LLM Summary
        try:
            log_print(f'\n🤖 Generating LLM summary...')
            hasil_llm = summarize_llm_analysis_batch(assessment_results)
            log_print(f'✅ LLM summary completed')
        except Exception as e:
            log_print(f'❌ ERROR in LLM summary: {str(e)}')
            log_print(f'   Traceback: {traceback.format_exc()}')
            hasil_llm = {
                "kesimpulan_llm": f"Error: {str(e)}",
                "rata_rata_confidence_score": 0,
                "error": str(e)
            }

        log_print(f'\n{"="*70}')
        log_print(f'✅ ALL AGGREGATE ANALYSIS COMPLETED')
        log_print(f'{"="*70}')

        # ============================================================================
        # SAVE JSON
        # ============================================================================
        if assessment_results:
            try:
                log_print(f'\n💾 SAVING JSON RESULTS...')

                results_json = {
                   "success": True,
                    "name": candidate_name,
                    "session": session_id,
                    "llm_results": hasil_llm,
                    "content": assessment_results,
                    "aggregate_cheating_analysis": aggregate_cheating,
                    "aggregate_non_verbal_analysis": aggregate_non_verbal,
                    "metadata": {
                        "total_videos": len(uploaded_videos),
                        "successful_videos": len(assessment_results),
                        "processed_at": datetime.now(timezone.utc).isoformat(),
                        "model": "faster-whisper large-v3",
                        "llm_model": "meta-llama/Llama-3.1-8B-Instruct"
                    }
                }

                results_filename = f"{session_id}.json"
                results_path = os.path.join(RESULTS_DIR, results_filename)

                log_print(f'📂 Results path: {results_path}')
                log_print(f'📊 JSON size: {len(str(results_json))} chars')

                # Ensure directory exists
                os.makedirs(RESULTS_DIR, exist_ok=True)
                log_print(f'✅ Results directory ensured: {RESULTS_DIR}')

                # Write JSON
                with open(results_path, 'w', encoding='utf-8') as f:
                    json.dump(results_json, f, ensure_ascii=False, indent=2)

                log_print(f'✅ JSON written to file')

                # Verify
                if os.path.exists(results_path):
                    file_size = os.path.getsize(results_path)
                    log_print(f'✅✅✅ JSON FILE SAVED SUCCESSFULLY! ✅✅✅')
                    log_print(f'   Path: {results_path}')
                    log_print(f'   Size: {file_size} bytes')
                else:
                    log_print(f'❌❌❌ WARNING: JSON FILE NOT CREATED! ❌❌❌')

                results_url = f"{base_url}/results/{results_filename}"
                log_print(f'🌐 Results URL: {results_url}')

            except Exception as e:
                log_print(f'❌ CRITICAL ERROR saving JSON: {str(e)}')
                log_print(f'   Traceback: {traceback.format_exc()}')
        else:
            log_print(f'\n⚠️⚠️⚠️ WARNING: assessment_results is EMPTY! ⚠️⚠️⚠️')
            log_print(f'   JSON will NOT be saved.')

        successful_count = sum(1 for t in transcriptions if 'transcription' in t)

        with processing_lock:
            processing_status[session_id] = {
                'status': 'completed',
                'result': {
                    'success': True,
                    'transcriptions': transcriptions,
                    'processed_videos': len(transcriptions),
                    'successful_videos': successful_count,
                    'failed_videos': len(transcriptions) - successful_count,
                    'results_url': f"{base_url}/results/{session_id}.json" if assessment_results else None
                }
            }

        log_print(f'\n{"="*70}')
        log_print(f'✅ SESSION COMPLETED')
        log_print(f'   Success: {successful_count}/{len(transcriptions)} videos')
        log_print(f'   Log file: {log_file}')
        log_print(f'{"="*70}\n')

    except Exception as e:
        log_print(f'\n❌ SESSION ERROR:\n{traceback.format_exc()}')

        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': traceback.format_exc()
            }

    finally:
        log_handle.close()

<b><h2> ENDPOINT

In [ ]:
# ENDPOINTS
@app.post('/upload')
async def receive_videos_and_process(
    request: Request,
    candidate_name: str = Form(...),
    language: str = Form("en"),
    videos: List[UploadFile] = File(...),
    questions: List[str] = Form(...)  # NEW: Accept questions array
):
    """Upload videos and start background transcription"""
    session_id = uuid.uuid4().hex
    print(f'\n🔵 NEW UPLOAD REQUEST - Session: {session_id}')
    print(f'   Candidate: {candidate_name}')
    print(f'   Videos: {len(videos)} file(s)')
    print(f'   Questions: {len(questions)} question(s)')  # NEW

    # NEW: Validate questions count matches videos count
    if len(questions) != len(videos):
        return JSONResponse(
            content={
                'success': False,
                'error': f'Questions count ({len(questions)}) must match videos count ({len(videos)})'
            },
            status_code=400,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    if language not in ["en", "id"]:
        return JSONResponse(
            {
                'success': False,
                'error': f'Invalid language code: {language}. Must be "en" or "id".'
            },
            status_code=400
        )

    # Initialize status FIRST
    with processing_lock:
        processing_status[session_id] = {
            'status': 'uploading',
            'progress': '0/0',
            'message': 'Uploading videos...'
        }

    try:
        # 1. Upload semua video (fast)
        base_url = str(request.base_url).rstrip('/')
        uploaded_videos = []

        print(f'\n📤 Uploading {len(videos)} video(s)...')
        for idx, (video, question) in enumerate(zip(videos, questions), 1):  # NEW: zip with questions
            try:
                ext = os.path.splitext(video.filename)[1] or '.webm'
                safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
                dest_path = os.path.join(UPLOAD_DIR, safe_name)

                # Update upload progress
                with processing_lock:
                    processing_status[session_id]['message'] = f'Uploading video {idx}/{len(videos)}...'
                    processing_status[session_id]['progress'] = f'{idx}/{len(videos)}'

                with open(dest_path, 'wb') as buffer:
                    shutil.copyfileobj(video.file, buffer)

                file_url = f"{base_url}/uploads/{safe_name}"
                uploaded_videos.append({
                    'positionId': idx,
                    'question': question,  # NEW: Include question
                    'isVideoExist': True,
                    'recordedVideoUrl': file_url,
                    'filename': safe_name
                })
                print(f'   ✅ Uploaded: {safe_name} | Q: {question[:50]}{"..." if len(question) > 50 else ""}')  # NEW

            except Exception as e:
                print(f'   ❌ Failed: {str(e)}')
                uploaded_videos.append({
                    'positionId': idx,
                    'question': question if idx <= len(questions) else '',  # NEW: Include question even on error
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': str(e)
                })

        # 2. Update status to processing
        with processing_lock:
            processing_status[session_id] = {
                'status': 'processing',
                'progress': '0/' + str(len(uploaded_videos)),
                'message': 'Starting transcription...',
                'uploaded_videos': len(uploaded_videos)
            }

        # 3. Start background thread
        thread = th.Thread(
            target=process_transcriptions_sync,
            args=(session_id, candidate_name, uploaded_videos, base_url, language),
            daemon=True
        )
        thread.start()

        print(f'✅ Upload complete. Background thread started.')
        print(f'📤 Returning immediate response with session_id: {session_id}')

        # 4. RETURN IMMEDIATELY - no waiting!
        return JSONResponse(
            content={
                'success': True,
                'session_id': session_id,
                'message': 'Videos uploaded successfully. Processing started.',
                'uploaded_videos': len(uploaded_videos)
            },
            status_code=200,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    except Exception as e:
        error_detail = traceback.format_exc()
        print(f'❌ Error:\n{error_detail}')

        # Update status to error
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': error_detail
            }

        return JSONResponse(
            content={
                'success': False,
                'session_id': session_id,
                'error': str(e)
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

In [ ]:
# ===== ENDPOINT: /upload_json =====
@app.post('/upload_json')
async def receive_json_and_download_videos(request: Request):
    """Receive JSON with Google Drive URLs, download videos, then process"""
    session_id = uuid.uuid4().hex
    
    try:
        # Parse JSON
        json_data = await request.json()
        
        print(f'\n🔵 NEW JSON UPLOAD REQUEST - Session: {session_id}')
        
        # Validate structure
        if not json_data.get('success') or not json_data.get('data'):
            return JSONResponse(
                {'success': False, 'error': 'Invalid JSON: missing success or data'},
                status_code=400,
                headers={'Access-Control-Allow-Origin': '*'}
            )
        
        data = json_data['data']
        
        # Extract candidate
        if not data.get('candidate') or not data['candidate'].get('name'):
            return JSONResponse(
                {'success': False, 'error': 'Missing candidate name'},
                status_code=400,
                headers={'Access-Control-Allow-Origin': '*'}
            )
        
        candidate_name = data['candidate']['name']
        candidate_email = data['candidate'].get('email', 'N/A')
        
        # Extract interviews
        if not data.get('reviewChecklists') or not data['reviewChecklists'].get('interviews'):
            return JSONResponse(
                {'success': False, 'error': 'Missing interviews data'},
                status_code=400,
                headers={'Access-Control-Allow-Origin': '*'}
            )
        
        interviews = data['reviewChecklists']['interviews']
        
        if not isinstance(interviews, list) or len(interviews) == 0:
            return JSONResponse(
                {'success': False, 'error': 'Interviews array is empty'},
                status_code=400,
                headers={'Access-Control-Allow-Origin': '*'}
            )
        
        # Get language
        language = json_data.get('language', 'en')
        
        print(f'   Candidate: {candidate_name} ({candidate_email})')
        print(f'   Videos: {len(interviews)} video(s)')
        print(f'   Language: {language}')
        
        # Validate language
        if language not in ["en", "id"]:
            return JSONResponse(
                {'success': False, 'error': f'Invalid language: {language}'},
                status_code=400,
                headers={'Access-Control-Allow-Origin': '*'}
            )
        
        # Log certification info
        if data.get('certification'):
            cert = data['certification']
            print(f'   Certification: {cert.get("abbreviatedType", "N/A")} - {cert.get("status", "N/A")}')
        
        # Initialize status
        with processing_lock:
            processing_status[session_id] = {
                'status': 'downloading',
                'progress': '0/' + str(len(interviews)),
                'message': 'Downloading videos from Google Drive...'
            }
        
        # Start background thread
        thread = th.Thread(
            target=download_and_process_videos,
            args=(session_id, candidate_name, interviews, language, str(request.base_url).rstrip('/')),
            daemon=True
        )
        thread.start()
        
        print(f'✅ JSON received. Background download thread started.')
        print(f'📤 Returning immediate response with session_id: {session_id}')
        
        return JSONResponse(
            content={
                'success': True,
                'session_id': session_id,
                'message': 'JSON received. Downloading videos from Google Drive...',
                'video_count': len(interviews)
            },
            status_code=200,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )
    
    except Exception as e:
        error_detail = traceback.format_exc()
        print(f'❌ Error processing JSON:\n{error_detail}')
        
        return JSONResponse(
            content={'success': False, 'error': str(e)},
            status_code=500,
            headers={'Access-Control-Allow-Origin': '*'}
        )

In [ ]:
@app.get('/status/{session_id}')
async def get_processing_status(session_id: str):
    """Check processing status"""
    with processing_lock:
        if session_id not in processing_status:
            return JSONResponse(
                {
                    'status': 'not_found',
                    'message': 'Session not found'
                },
                status_code=404,
                headers={
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'GET, OPTIONS',
                    'Access-Control-Allow-Headers': '*',
                    'Cache-Control': 'no-cache, no-store, must-revalidate',
                }
            )

        status_copy = processing_status[session_id].copy()

    # Add redirect URL if completed
    if status_copy.get('status') == 'completed':
        status_copy['redirect'] = f"halaman_dasboard.html?session={session_id}"

    return JSONResponse(
        status_copy,
        headers={
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'GET, OPTIONS',
            'Access-Control-Allow-Headers': '*',
            'Cache-Control': 'no-cache, no-store, must-revalidate',
        }
    )


In [ ]:
@app.get('/results/{session_id}')
async def get_results(session_id: str):
    """Get assessment results for a session"""
    results_filename = f"{session_id}.json"
    results_path = os.path.join(RESULTS_DIR, results_filename)

    if not os.path.exists(results_path):
        return JSONResponse(
            {
                'success': False,
                'message': 'Results not found for this session',
                'session_id': session_id
            },
            status_code=404,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    try:
        with open(results_path, 'r', encoding='utf-8') as f:
            results_data = json.load(f)

        return JSONResponse(
            results_data,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
                'Cache-Control': 'no-cache, no-store, must-revalidate',
            }
        )
    except Exception as e:
        return JSONResponse(
            {
                'success': False,
                'message': f'Error reading results: {str(e)}',
                'session_id': session_id
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )


In [ ]:
@app.get('/')
async def index():
    return {
        'message': 'AI Interview Assessment System',
        'model': 'faster-whisper large-v3',
        'accuracy': '98%+ for clear English speech',
        'speed': '4-5x faster than standard Whisper',
        'endpoints': {
            'upload': 'POST /upload',
            'status': 'GET /status/{session_id}',
            'results': 'GET /results/{session_id}',
            'test_form': 'GET /upload_form'
        }
    }

<b><h2> LOCAL SERVER

In [ ]:
# Jalankan server uvicorn di dalam notebook (tanpa ngrok)
nest_asyncio.apply()
PORT = 8888

# Hentikan server sebelumnya jika ada
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        # Tunggu thread selesai (dengan timeout)
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Buat server instance baru dengan log level yang lebih rendah
config = uvicorn.Config(
    app=app,
    host='0.0.0.0',
    port=PORT,
    log_level='warning',  # Kurangi verbosity untuk menghindari duplikasi log
    access_log=False  # Nonaktifkan access log di console
)
server = uvicorn.Server(config=config)

# Fungsi untuk menjalankan server di thread
def run_server_in_thread():
    # Buat event loop baru untuk thread ini
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

# Jalankan server di background thread
server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

print('━' * 60)
print('🚀 Server started successfully!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'📍 Network URL: http://0.0.0.0:{PORT}')
print(f'🔧 Endpoints:')
print(f'   - POST /upload       (upload videos & process)')
print(f'   - POST /upload_json  (upload JSON & download videos)')
print(f'   - GET  /status/{{id}}  (check processing status)')
print(f'   - GET  /results/{{id}} (get assessment results)')
print(f'   - GET  /upload_form  (test form)')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('━' * 60)

<b><h2> NGROK

In [ ]:
# Configure ngrok
# Set ngrok authtoken (dapatkan dari https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = getpass.getpass('Enter your ngrok authtoken: ')
conf.get_default().auth_token = NGROK_AUTH_TOKEN

print('✅ Ngrok configured successfully')

In [ ]:
# Start server with ngrok
nest_asyncio.apply()
PORT = 8888

# Stop previous server if exists
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Close previous ngrok tunnels
try:
    ngrok.kill()
except:
    pass

# Create server instance
config = uvicorn.Config(
    app=app,
    host='0.0.0.0',
    port=PORT,
    log_level='warning',
    access_log=False
)
server = uvicorn.Server(config=config)

# Run server in thread
def run_server_in_thread():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(PORT, bind_tls=True)
ngrok_url = public_url.public_url

print('┏' + '━' * 70 + '┓')
print('🚀 Server started successfully with ngrok!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'🌐 Public URL (ngrok): {ngrok_url}')
print(f'📋 Copy this URL to use in Upload.js:')
print(f'   const VIDEO_ENDPOINT = "{ngrok_url}/upload";')
print(f'📧 Endpoints:')
print(f'   - POST {ngrok_url}/upload')
print(f'   - POST {ngrok_url}/upload_json')
print(f'   - GET  {ngrok_url}/status/{{id}}')
print(f'   - GET  {ngrok_url}/results/{{id}}')
print(f'   - GET  {ngrok_url}/upload_form')
print('ℹ️  Ngrok tunnel will stay active while notebook is running')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('┗' + '━' * 70 + '┛')

In [ ]:
# Cari session_id terbaru
log_files = [f for f in os.listdir('.') if f.startswith('session_') and f.endswith('.log')]
if log_files:
    latest_log = max(log_files, key=lambda x: os.path.getmtime(x))
    print(f"📝 Reading log: {latest_log}\n")
    print("="*70)

    with open(latest_log, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content)

    print("="*70)
else:
    print("⚠️ No log files found")

## System Information

### Whisper Model
- **Library**: `faster-whisper` (optimized implementation)
- **Model**: `large-v3` (most accurate available)
- **Accuracy**: ~98% for clear English speech
- **Speed**: 4-5x faster than `openai-whisper`

### Translation
- **Provider**: DeepL API
- **Target Language**: Indonesian (ID)
- **Source Language**: English (EN)
- **Character Limit**: 5,000 per chunk
- **Setup**: Set `DEEPL_API_KEY` in cell 4
- **Get API Key**: https://www.deepl.com/pro-api (Free tier: 500,000 chars/month)

### LLM Assessment
- **Model**: meta-llama/Llama-2-7b-chat-hf
- **Method**: Hybrid (LLM + Static)
- **LLM Evaluated Criteria** (3):
  1. **Kualitas Jawaban** - Quality of answer (clarity, completeness, depth)
  2. **Koherensi** - Coherence (logical flow, consistency, structure)
  3. **Relevansi** - Relevance (alignment with question, staying on topic)
- **Static Dummy Values** (2):
  4. **Tempo Bicara** - Speaking tempo (fixed at 85/100) 🔧 *TODO: Replace with audio analysis model*
  5. **Confidence Score** - Confidence (fixed at 82/100) 🔧 *TODO: Replace with voice analysis model*
- **Cheating Detection**: LLM analyzes for multiple speakers, artificial voice, reading patterns
- **Fallback**: Rule-based assessment if LLM fails

### Performance
- **Device**: Automatically detects CUDA GPU (if available) or CPU
- **Compute Type**:
  - GPU: `float16` (faster with high accuracy)
  - CPU: `int8` (optimized for CPU)
- **VAD Filter**: Enabled (skips silence for efficiency)

### Settings
- **Beam Size**: 5 (higher = more accurate)
- **Best Of**: 5 (samples multiple candidates)
- **Patience**: 2.0 (thorough beam search)
- **Temperature**: 0.0 (deterministic output)
- **Context**: Uses previous text for better accuracy

### Storage Management
- **Auto-delete videos**: ✅ Videos are automatically deleted after successful transcription
- **Storage saved**: Only transcriptions and results are kept
- **Safety**: Deletion only happens after successful transcription
- **Error handling**: If deletion fails, processing continues normally

### Endpoints
- `POST /upload` - Upload videos and start transcription
- `GET /status/{session_id}` - Check processing status
- **`GET /results/{session_id}`** - **Get assessment results**
- `GET /upload_form` - Test form interface
- `GET /` - System information

### Files
- ~~Uploaded videos: `uploads/`~~ (deleted after transcription) ♻️
- Transcriptions: `transcriptions/` ✅ (includes English + Indonesian + Assessment)
- **Assessment results: `results/`** ✅

### Assessment Data Structure
```json
{
  "success": true,
  "name": "Candidate Name",
  "session": "session_id_here",
  "content": [
    {
      "id": 1,
      "question": "What is your experience with Python?",
      "result": {
        "penilaian": {
          "kualitas_jawaban": 85,    // ✅ LLM evaluated
          "koherensi": 83,            // ✅ LLM evaluated
          "relevansi": 80,            // ✅ LLM evaluated
          "tempo_bicara": 85,         // 🔧 Static dummy (TODO: audio model)
          "confidence_score": 82,     // 🔧 Static dummy (TODO: voice model)
          "total": 83
        },
        "penilaian_akhir": 4,
        "cheating_detection": "Tidak",
        "keputusan_akhir": "Lulus",
        "transkripsi_en": "...",
        "transkripsi_id": "...",
        "metadata": {
          "assessment_method": "Hybrid (LLM + Static)",
          "llm_evaluated_criteria": ["kualitas_jawaban", "koherensi", "relevansi"],
          "static_criteria": ["tempo_bicara", "confidence_score"]
        }
      }
    }
  ],
  "metadata": {
    "assessment_method": "Hybrid (LLM + Static)",
    "llm_criteria": ["kualitas_jawaban", "koherensi", "relevansi"],
    "static_criteria": ["tempo_bicara", "confidence_score"]
  }
}
```

### Roadmap
- ✅ **Phase 1**: LLM Assessment (kualitas, koherensi, relevansi)
- 🔧 **Phase 2**: Audio Analysis Model (tempo_bicara) - *Coming Soon*
- 🔧 **Phase 3**: Voice Analysis Model (confidence_score) - *Coming Soon*
- 🔧 **Phase 4**: Video Analysis (eye contact, body language) - *Future*

### Notes
- **3 criteria** evaluated by LLM with real intelligence
- **2 criteria** use static dummy values (will be replaced with specialized models)
- Static values: `tempo_bicara=85`, `confidence_score=82`
- Results saved automatically after transcription completes
- **Original video files are deleted after transcription to save storage**
- DeepL API key required for translation (free tier available)
- Access via: `http://127.0.0.1:8888/results/{session_id}`

### DeepL Setup
1. Sign up at https://www.deepl.com/pro-api
2. Get your free API key (500,000 chars/month)
3. Set `DEEPL_API_KEY` in cell 4
4. Restart kernel and run all cells